<a href="https://colab.research.google.com/github/Issamara/Tarea2ARF/blob/main/Tarea2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext rpy2.ipython
!rm -rf RiesgosFinancieros
!git clone "https://github.com/pikey-msc/RiesgosFinancieros/"

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
Cloning into 'RiesgosFinancieros'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 649 (delta 51), reused 0 (delta 0), pack-reused 579
Receiving objects: 100% (649/649), 33.01 MiB | 24.69 MiB/s, done.
Resolving deltas: 100% (430/430), done.


In [ ]:
%%R
#require(quantmod)
install.packages("quantmod")
library(quantmod)
#require(data.table)
install.packages("data.table")
library(data.table)
#require("PerformanceAnalytics")
install.packages("PerformanceAnalytics")
library("PerformanceAnalytics")

install.packages("Deriv")
library(Deriv)


library(dplyr)
con = gzcon(url('https://github.com/systematicinvestor/SIT/raw/master/sit.gz', 'rb'))
source(con)
close(con)

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/quantmod_0.4.17.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 152204 bytes (148 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[wr

In [ ]:
%%R
#FUNCIÓN DE INTERPOLACIÓN ALAMBRADA

talamb=function(nodos,curva,plazos) #función de interpolación de tasas por el método alamabrada
{
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n)
  {
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}


#funciones necesarias
diagv=function(x)			#función para diagonalizar un vector
{
  n01=nrow(as.matrix(x))
  m01=ncol(as.matrix(x))
  dimmax=max(n01,m01)
  res=matrix(0,dimmax,dimmax)
  for (i in 1:dimmax)
  {
    res[i,i]=x[i]
  }
  res
}

#interpolación
talamb=function(nodos,curva,plazos) #función de interpolación de tasas por el método alamabrada
{
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n)
  {
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}


# Definición de parámetros para valorar:
Acciones
Bonos
Bonos M
Bondes D
SWAPS
Futuros
Opciones

Cargamos parámetros para valoración:

In [ ]:
%%R
fval=as.Date("20200228",format="%Y%m%d") #Fecha de valoración
itpl=0 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada
alpha=0.95 #Nivel de confianza para obtener estimaciones de riesgo
#setwd(direc)	

#ACCIONES Y DIVISAS
#Cargar los símbolos de yahoo finance para EQ
Symbols<-c("AMXL.MX", "GCARSOA1.MX", "WALMEX.MX" )#tienen que ir en orden alfabético
pos_eq=c(-5000,1000,1200) #monto inicial invertido en acciones
#Cargar los símbolos de yahoo finance para FX
SymbolsFX<-c("EURUSD=X","GBPUSD=X","USDMXN=X" ) #tienen que ir en orden alfabético
pos_fx=c(700,-600,1500) #monto inicial invertido en divisas (cantidad de divisas)
nh=3660 #días de historia

#CETES
base="RiesgosFinancieros/2021-1/Tarea/tasa_guber.txt"
plazos_bcc=c(180)
contratos_bcc=c(1500)
nominal_bcc=10 #CETE

#BONOS M
btasadesc_bm="RiesgosFinancieros/2021-1/Tarea/tasa_yield.txt"
tf_bm=c(0.065,0) #tasa fija
tfcupon_bm=c(0.025,0) #Tasafija del cupón
plazos_bm=c(3600,0) #Vencimiento del bono
plazocupon_bm=c(360,0) #plazos_bm fijos de cada cupón
contratos_bm=1 #posición invertida
nominal_bm=100

#BONDES D
btasadescst="RiesgosFinancieros/2021-1/Tarea/tasa_guber_st.txt"
btasafondeo="RiesgosFinancieros/2021-1/Tarea/tfondeo.txt"
plazos_bdm=707 #Vencimiento del bono
plazocupon_bdm=28 #plazos_bdm fijos de cada cupón
contratos_bdm=-1000 #posición invertida
nominal_bdm=100

#FORWARDS TDC
bext="RiesgosFinancieros/2021-1/Tarea/tasa_libor.txt"
bdom="RiesgosFinancieros/2021-1/Tarea/tasa_fwd.txt"
btsp="RiesgosFinancieros/2021-1/Tarea/tasa_spot.txt"
SymbolsFX_ftdc<-c("USDMXN=X", "GBPUSD=X" ) #tienen que ir en orden alfabético
plazos_fwd=cbind(5)
contratos_fwd=cbind(100 )
kst_fwd=cbind(19.83 )
nominal_fwd=1
yext=1 #si se carga información de yahoo en la fecha definida por fval o SymbolsFX, en caso contrario se utiliza información que se tendrá que cargar de tasas_spot.txt
trlib=1 #1 si la curva libor viene a 182 0 si no.

#FORWARDS DE IPC
#Descontamos con gubernamental
base2="RiesgosFinancieros/2021-1/Tarea/tasa_dividendos.txt"
SymbolsEQ_find<-c("^MXX", "GCARSOA1.MX" ) #tienen que ir en orden alfabético
plazos_fwd_ind=cbind(53)
contratos_fwd_ind=cbind(50)
kst_fwd_ind=cbind(49525)
nominal_fwd_ind=1

#SWAP

btasadesc_sw="RiesgosFinancieros/2021-1/Tarea/tasa_TIIE_SW_OP.txt"
btasacupvar_sw="RiesgosFinancieros/2021-1/Tarea/tasa_DIRS_SW_OP.txt"
tasafija_sw=cbind(0.052,0.049) #se establece la tasa fija a pagar para cada swap
plazos_sw=cbind(588,270) #se establece el número de días que vivirá el swap
plazocupon_sw=cbind(28,28) #se establece el número de días que se pagará cada cupón
contratos_sw=cbind(1600,-1200) #se establece el número de contratos_sw de cada swap (-1600,1200)
nominal_sw=cbind(1,1) #se establece el nominal_sw de cada swap (1,1)
por_sw=cbind(0,1) #se establece 0 si se paga tasa fija y 1 si se paga tasa variable (0,0)


#OPCIONES
btasadesc_oir="RiesgosFinancieros/2021-1/Tarea/tasa_TIIE_SW_OP.txt" 
btasaspot_oir="RiesgosFinancieros/2021-1/Tarea/tasa_DIRS_SW_OP.txt" 
bvolspot_oir="RiesgosFinancieros/2021-1/Tarea/tvoltiie_opc.txt"
plazos_oir=cbind(1700, 700) #T-t
pr_oir=28 #plazo de referencia  
dct_oir=360 #d_base
cp_oir=cbind(1,0) #si es call (cap) o put (floor) (1,0,0,0,0)
K_oir=cbind( 0.094, 0.089) #( 0.07, 0.085, 0.078, 0.085, 0.08)
contratos_oir=cbind(1000, 500) #Número de contratos (2200, -2900, 2900, -460, 5000)
nominal_oir=1
cs_oir=1 #1 si es continua la tasa 0 si es simple



# Carga de datos

In [ ]:
%%R
#CARGA DE DATOS DE ACCIONES
pos=cbind(t(pos_fx),t(pos_eq))
start_date=Sys.Date()-nh #fecha inicial
#Creación del objeto para guardar los datos
dataEnv<-new.env()
dataEnvFX<-new.env()
#obtener los datos
getSymbols.yahoo(Symbols,env=dataEnv,from=start_date)
getSymbols.yahoo(SymbolsFX,env=dataEnvFX,from=start_date)
#muestra
#tail(dataEnvFX$`GBPUSD=X`)
#limpiarlos, alinearnos y quedarnos con el precio de cierre 
bt.prep(dataEnv,align='remove.na',fill.gaps = T)
bt.prep(dataEnvFX,align='remove.na',fill.gaps=T)
#muestra de datos
#head(dataEnv$prices)
#head(dataEnvFX$prices)
#Nos quedamos con los precios
stock_prices = dataEnv$prices
#tail(stock_prices[,])
stock_pricesFX=dataEnvFX$prices
#tail(stock_pricesFX)
#cambiar todo a pesos mexicanos
stock_pricesFX=cbind(stock_pricesFX[,1,with=F]*stock_pricesFX[,3,with=F],stock_pricesFX[,2,with=F]*stock_pricesFX[,3,with=F],stock_pricesFX[,3,with=F])
#tail(stock_pricesFX)
stock_prices_EQFX=merge(stock_pricesFX,stock_prices,join = "inner")
#stock_prices_EQFX
#tail(stock_prices_EQFX)
#Preciso actuales
#x0=as.data.table(as.matrix(stock_prices_EQFX[nrow(stock_prices_EQFX),])) #valores actuales
x0=stock_prices_EQFX[nrow(stock_prices_EQFX),]
#x0

aux2=data.table(Date=as.Date(index(stock_prices_EQFX)),coredata(stock_prices_EQFX))



#LEER DATOS DE CETES #CARGA DE DATOS DE BONO CUPÓN CERO
data<-read.table(base)
n<-nrow(data)
m_gov=ncol(data)
#x_orig_gov=data.frame(data[2:n,1:m_gov])
x_orig_gov=as.data.table(mutate(data[2:n,1:m_gov],Date=as.Date(V1,format="%Y%m%d")))
x_orig_gov=x_orig_gov%>%select(-V1)
nodos_gov=data.frame(data[1,2:m_gov])

#LEER DATOS DE BONOS M
#carga de datos
#carga de tasas de descuento
data1<-read.table(btasadesc_bm)
n<-nrow(data1)
m_tybm=ncol(data1)
X_orig_tybm=as.data.table(mutate(data1[2:n,1:m_tybm],Date=as.Date(V1,format="%Y%m%d")))
X_orig_tybm=X_orig_tybm%>%select(-V1)
nodos_tybm=data.frame(data1[1,2:m_tybm])


##CARGA DE DATOS DE BONDE D
#carga de datos
#carga de tasas de descuento
data1<-read.table(base)
n<-nrow(data1)
m_bd=ncol(data1)
##X_orig_bd=as.data.table(mutate(data1[2:n,1:m_tybm],Date=as.Date(V1,format="%Y%m%d")))
#X_orig_bd%>%select(-V1)

X1_orig=mutate(data.frame(data1[2:n,1:m_bd]), V1=as.Date(V1,format="%Y%m%d"))
#nodos=data.frame(data1[1,2:m_bd])
n=n-1

data3<-read.table(btasadescst)
n3<-nrow(data3)
m3_bd=ncol(data3)
X3_orig_bd=as.data.table(mutate(data3[2:n,1:m3_bd],Date=as.Date(V1,format="%Y%m%d")))
X3_orig_bd=X3_orig_bd%>%select(-V1)

X3a_orig=mutate(data.frame(data1[2:n,1:m_bd]), V1=as.Date(V1,format="%Y%m%d"))
nodos3_bd=data.frame(data3[1,2:m3_bd])
n3=n3-1

data2<-read.table(btasafondeo)
n2<-nrow(data2)
X2_orig_bd=data.frame(data2[2:n2,1:2])
X2_orig_bd=mutate(X2_orig_bd, V1=as.Date(V1,format="%Y%m%d"), Date=as.Date(V1,format="%Y%m%d"), V2=as.numeric(as.character(V2)))
tfh=seq(min(X2_orig_bd$V1), max(X2_orig_bd$V1), "days") #sucesión de dias para tasa fondeo
tfhd=data.frame(ID=1:count(tfh),fecha=tfh)

#Cruzar la sucesión de todos los días versus el de tasa de fondeo
tfhd=setDT(tfhd)[, Date := tfh][order(-Date)]
X2_orig_bd=setDT(X2_orig_bd)[, Date := V1][order(-Date)]
# rolling join unión por rolling, rellena las fechas que faltaban con el último valor conocido "roll=Inf"
X2_orig_bd=X2_orig_bd[tfhd, on = .(Date), roll = Inf] 
#buscar fecha de valuación en tfondeo
tf_act=X2_orig_bd[fecha==fval,]$V2/100
tf_int=X2_orig_bd[fecha<=fval & fecha>=(fval-plazocupon_bdm[1])]$V2/100

X1_orig=setDT(X1_orig)[, Date:= V1][order(-Date)] #Para alinear con valor presente y tasa de fondeo.


#CARGA DE DATOS DE  FORWARDS DE TDC
#datas
#data<-read.table("tasa_tiie.txt")
data1<-read.table(bext)
data2<-read.table(bdom)

########minimos para parametrizar
n1=nrow(data1)
n2=nrow(data2)
m1_ftdc=ncol(data1)
m2_ftdc=ncol(data2)
n=min(n1,n2)-1
###NODOS###
nodos1_ftdc=data.frame(data1[1,2:m1_ftdc])
nodos2_ftdc=data.frame(data2[1,2:m2_ftdc])
####MATRICES DEL MISMO TAMAÑO MENOS DOLAR

x1_ftdc=as.data.table(mutate(data1[2:n,1:m1_ftdc],Date=as.Date(V1,format="%Y%m%d")))
x1_ftdc=x1_ftdc%>%select(-V1)
x2_ftdc=as.data.table(mutate(data2[2:n,1:m2_ftdc],Date=as.Date(V1,format="%Y%m%d")))
x2_ftdc=x2_ftdc%>%select(-V1)

###Para Dolar

if (yext==1)
{
  #Cargar los símbolos de yahoo finance para FX
  start_date=fval-3660 #fecha inicial
  
  #Creación del objeto para guardar los datos
  dataEnvFX<-new.env()
  
  #obtener los datos
  getSymbols.yahoo(SymbolsFX_ftdc,env=dataEnvFX,from=start_date, to=(fval))
  #limpiarlos, alinearnos y quedarnos con el precio de cierre 
  bt.prep(dataEnvFX,align='remove.na',fill.gaps=T)
  
  #muestra de datos
  head(dataEnvFX$prices[,2])
  
  #Nos quedamos con los precios
  X3_ftdc=data.table(Date=as.Date(index(dataEnvFX$prices[,2])),coredata(dataEnvFX$prices[,2]))
} else  
{ 
data3<-read.table(btsp)
print(head(data3))
n3<-nrow(data3)
m3<-ncol(data3)
X3=data.table(as.matrix(as.double(as.matrix(data3[2:(n+1),m3]))))
X3_find=as.data.table(mutate(data3[2:(n+1),1:m3],Date=as.Date(V1,format="%Y%m%d")))
}


# CARGA DE DATOS DE  FORWARD DE IPC

data<-read.table(base)
data_1<-read.table(base2) #tasa dividendos
n<-nrow(data)
m_gov=ncol(data)
m2_div=ncol(data_1)
#x_orig_gov=data.frame(data[2:n,1:m_gov])
x_orig_gov=as.data.table(mutate(data[2:n,1:m_gov],Date=as.Date(V1,format="%Y%m%d")))
x_orig_gov=x_orig_gov%>%select(-V1)
x2_fdiv=as.data.table(mutate(data_1[2:n,1:m2_div],Date=as.Date(V1,format="%Y%m%d")))
x2_fdiv=x2_fdiv%>%select(-V1)
nodos_gov=data.frame(data[1,2:m_gov])
nodos2_div=data.frame(data_1[1,2:m2_div])

  #Cargar los símbolos de yahoo finance para EQ
  start_date=fval-nh #fecha inicial
  
  #Creación del objeto para guardar los datos
  dataEnvEQ<-new.env()
  
  #obtener los datos
  getSymbols.yahoo(SymbolsEQ_find,env=dataEnvEQ,from=start_date, to=(fval))
  #limpiarlos, alinearnos y quedarnos con el precio de cierre 
  bt.prep(dataEnvEQ,align='remove.na',fill.gaps=T)
  
  #muestra de datos
#  head(dataEnvEQ$prices)
  
  #Nos quedamos con los precios
  X3_find=data.table(Date=as.Date(index(dataEnvEQ$prices[,2])),coredata(dataEnvEQ$prices[,2]))

  
  #CARGA DE DATOS PARA SWAP
  data1<-read.table(btasadesc_sw)
  n1<-nrow(data1)
  m1_orig_sw=ncol(data1)
  X1_orig_sw=data.table(mutate(data1[2:n1,1:m1_orig_sw],Date=as.Date(V1,format="%Y%m%d")))
  X1_orig_sw=X1_orig_sw%>%select(-V1)
  nodos1_sw=data.frame(data1[1,2:m1_orig_sw])
  
  data2<-read.table(btasacupvar_sw)
  n2<-nrow(data2)
  m2_orig_sw=ncol(data2)
  X2_orig_sw=data.table(mutate(data2[2:n1,1:m1_orig_sw],Date=as.Date(V1,format="%Y%m%d")))
  X2_orig_sw=X2_orig_sw%>%select(-V1)
  nodos2_sw=data.frame(data2[1,2:m2_orig_sw])
  
  
  #CARGA DE DATOS PARA OPCIONES DE TASA DE INTERÉS
  #carga de datos
  #carga de rho	
  data1<-read.table(btasadesc_oir) 
  n<-nrow(data1)

  m1_orig_oir=ncol(data1)
  x1_orig_oir=data.table(mutate(data1[2:n,1:m1_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x1_orig_oir=x1_orig_oir%>%select(-V1)
  nodos1_oir=data.frame(data1[1,2:m1_orig_oir])
  
  #data1[1:3,]
  #carga de tasas spot
  data2<-read.table(btasaspot_oir)
  n<-nrow(data2)
  m2_orig_oir=ncol(data2)
  x2_orig_oir=data.table(mutate(data2[2:n,1:m2_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x2_orig_oir=x2_orig_oir%>%select(-V1)
  nodos2_oir=data.frame(data2[1,2:m2_orig_oir])
  #carga de volatilidades de spot
  data3<-read.table(bvolspot_oir)
  n<-nrow(data3)
  m3_orig_oir=ncol(data3)
  x3_orig_oir=data.table(mutate(data3[2:n,1:m3_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x3_orig_oir=x3_orig_oir%>%select(-V1)
  
  nodos3_oir=data.frame(data3[1,2:m3_orig_oir])



# Integración de Insumos

Aseguramiento de que todos los factores de riesgo estén alineados en sus fechas, nos quedamos con la intersección. Posteriormente, filtramos cada histórico factor de riesgo hacia esta intersección de todos.

In [ ]:
%%R
#INTERSECCIÓN DE FECHAS DE TODOS LOS INSUMOS    
  
lin_gub=data.table(Date=as.Date(aux2[x_orig_gov,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity y guber
lin_gub=unique(lin_gub)
lin_gub_bmy=data.table(Date=as.Date(lin_gub[X_orig_tybm,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity y guber, con bonos M
lin_gub_bmy=unique(lin_gub_bmy)
lin_gub_bmybdst=data.table(Date=as.Date(lin_gub_bmy[X3_orig_bd,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber y st (bonde)
lin_gub_bmybdst=unique(lin_gub_bmybdst)
lin_gub_bmybdst_flib=data.table(Date=as.Date(lin_gub_bmybdst[x1_ftdc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor
lin_gub_bmybdst_flib=unique(lin_gub_bmybdst_flib)
lin_gub_bmybdst_flibfwd=data.table(Date=as.Date(lin_gub_bmybdst_flib[x2_ftdc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd
lin_gub_bmybdst_flibfwd=unique(lin_gub_bmybdst_flibfwd)
lin_gub_bmybdst_flibfwdsp=data.table(Date=as.Date(lin_gub_bmybdst_flibfwd[X3_ftdc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot
lin_gub_bmybdst_flibfwdsp=unique(lin_gub_bmybdst_flibfwdsp)
lin_gub_bmybdst_flibfwdspind=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdsp[X3_find,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index 
lin_gub_bmybdst_flibfwdspind=unique(lin_gub_bmybdst_flibfwdspind)
lin_gub_bmybdst_flibfwdspind_swcup=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind[X1_orig_sw,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón
lin_gub_bmybdst_flibfwdspind_swcup=unique(lin_gub_bmybdst_flibfwdspind_swcup)
lin_gub_bmybdst_flibfwdspind_swcupvp=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcup[X2_orig_sw,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón
lin_gub_bmybdst_flibfwdspind_swcupvp=unique(lin_gub_bmybdst_flibfwdspind_swcupvp)
lin_gub_bmybdst_flibfwdspind_swcupvp_oirs=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp[x1_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot
lin_gub_bmybdst_flibfwdspind_swcupvp_oirs=unique(lin_gub_bmybdst_flibfwdspind_swcupvp_oirs)
lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp_oirs[x2_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot, opc ir vp
lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp=unique(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp)
lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol1=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp[x3_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot, opc ir vp, opc ir vol
lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol1[x2_fdiv,on=.(Date),nomatch=0]$Date))
lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol=unique(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol)

n=nrow(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol) #Historia de todos
print(n)

#historia de acciones y divisas
stock_prices_EQFX=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[aux2,on=.(Date),nomatch=0][order(-Date)]
stock_prices_EQFX=stock_prices_EQFX%>%select(-Date)

#historia de curva gubernamental
x_orig_gov=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x_orig_gov,on=.(Date),nomatch=0][order(-Date)]
x_orig_gov=x_orig_gov%>%select(-Date)

#historia de curva yield
X_orig_tybm=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X_orig_tybm,on=.(Date),nomatch=0][order(-Date)]
X_orig_tybm=X_orig_tybm%>%select(-Date)


#Historia de curvas de bonde
#CONSIDERAR LA CURVA GUBERNAMENTAL X1_ORIG_GOV
X3_orig_bd=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X3_orig_bd,on=.(Date),nomatch=0][order(-Date)]
X3_orig_bd=X3_orig_bd%>%select(-Date)


#historia de curvas de forward tdc
x1_ftdc=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x1_ftdc,on=.(Date),nomatch=0][order(-Date)]
x1_ftdc=x1_ftdc%>%select(-Date)/100
x2_ftdc=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x2_ftdc,on=.(Date),nomatch=0][order(-Date)]
x2_ftdc=x2_ftdc%>%select(-Date)/100
X3_ftdc=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X3_ftdc,on=.(Date),nomatch=0][order(-Date)]
X3_ftdc=X3_ftdc%>%select(-Date)

#historia de curvas de forward ind
#CONSIDERAR LA CURVA GUBERNAMENTAL X1_ORIG_GOV
X3_find=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X3_find,on=.(Date),nomatch=0][order(-Date)]
X3_find=X3_find%>%select(-Date)

#historia de swaps
X1_orig_sw=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X1_orig_sw,on=.(Date),nomatch=0][order(-Date)]
X1_orig_sw=X1_orig_sw%>%select(-Date)/100
X2_orig_sw=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X2_orig_sw,on=.(Date),nomatch=0][order(-Date)]
X2_orig_sw=X2_orig_sw%>%select(-Date)/100

#historia de opciones
x1_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x1_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x1_orig_oir=x1_orig_oir%>%select(-Date)/100
x2_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x2_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x2_orig_oir=x2_orig_oir%>%select(-Date)/100
x3_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x3_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x3_orig_oir=x3_orig_oir%>%select(-Date)



[1] 151


# Pasos para medir el riesgo por el método de Delta-Gamma

Para todos los instrumentos hay que definir y calcular los siguientes elementos:

1.Historico de factores de riesgo\
2.Vector de precios actual\
3.Derivada con respecto al tiempo evaluada en el precio actual\
4.Primera derivada con respecto a factores de riesgo evaluada y multiplicada con su precio actual\
5.Segunda derivada con respecto a factores de riesgo evaluada y multiplicada con sus precios actuales (al cuadrado)\

## Acciones y Divisas



In [ ]:
%%R
#divisas y acciones CÁLCULO
x0_acc_div=stock_prices_EQFX[1,]
DeltaX_acc_div=as.matrix(log(as.matrix(stock_prices_EQFX[1:(n-1)])/as.matrix(stock_prices_EQFX[2:(n)])))
V0_acc_div=cbind(t(pos_fx),t(pos_eq))*x0_acc_div
mu_acc_div=0
g_acc_div=cbind(t(pos_fx),t(pos_eq))
g_acc_divt=cbind(t(pos_fx),t(pos_eq))*x0_acc_div
gamma_acc_divt=matrix(0,ncol(x0_acc_div),ncol(x0_acc_div))

m_fx=length(pos_fx)
m_acc=length(pos_eq)



## CETES

In [ ]:
%%R

#CETE CÁLCULO
#paso para interpolar las tasas
m=length(plazos_bcc)
print(m)
x_bcc=matrix(0,n,m)
for (i in 1:(n))
{
  #x[i,]=approx(nodos_gov,x_orig_gov[i,],plazos_bcc)$y
  x_bcc[i,]=if(itpl==0){approx(nodos_gov,x_orig_gov[i,],plazos_bcc,rule=2)$y}else{talamb(nodos_gov,x_orig_gov[i,],plazos_bcc)}
}

x0_bcc=x_bcc[1,]

#función
bonocupcero = function(i,t)
{
  1/(1+i*t/360)
}

#funcion primera derivada con tiempo
bcc_dt = function(i,t)
{
  -(i/360)/(1+i*t/360)^2
}

#funcion primera derivada con tasa interés
bcc_di = function(i,t)
{
  -(t/360)/(1+i*t/360)^2
}

#funcion segunda derivada con tasa de interés
bcc_di2 = function(i,t)
{
  2*(t/360)^2/(1+i*t/360)^3
}

V0_bcc=as.matrix(bonocupcero(x0_bcc,plazos_bcc))*contratos_bcc*nominal_bcc #Valor actual de cada bono

VT0_bcc=sum(V0_bcc) #Valor total del portafolio al tiempo 0

mu_bcc=bcc_dt(x0_bcc,plazos_bcc)*contratos_bcc*nominal_bcc
g_bcc=bcc_di(x0_bcc,plazos_bcc)*contratos_bcc*nominal_bcc
g_bcct=g_bcc*x0_bcc
gamma_bcc=bcc_di2(x0_bcc,plazos_bcc)*contratos_bcc*nominal_bcc
gamma_bcct2=diagv(gamma_bcc*(x0_bcc)^2)



[1] 1


In [ ]:
%%R
x0_bcc

[1] 0.08097692


## Bonos M

In [ ]:
%%R
#BONO M  CÁLCULO
#Posición inicial
#interpolación de tasas y volatilidades
m=length(plazos_bm)
X_bm=matrix(0,n,m)
for (i in 1:(n))
{
  X_bm[i,]=if(itpl==0){approx(nodos_tybm,X_orig_tybm[i,],plazos_bm,rule=2)$y}else{talamb(nodos_tybm,X_orig_tybm[i,],plazos_bm)}
}


#Función de valoración por tasa yield
bonoMyield=function(x, plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm) #valoración bono tasa fija
{
  N=as.integer(plazos_bm/plazocupon_bm)+1   
  p1=plazos_bm-plazocupon_bm*(N-1)   
  a=(1-(1+x*plazocupon_bm/360)^(-N))/(plazocupon_bm*x/360)
  ((contratos_bm*nominal_bm*tfcupon_bm*plazocupon_bm/360)*a+(contratos_bm*nominal_bm)/((1+x*plazocupon_bm/360)^N))*(1+x*plazocupon_bm/360)^(1-p1/plazocupon_bm)   
}

x0_bm=t(as.matrix(X_bm[1,])) #tasas de descuento valor actual
V0_bm=bonoMyield(x0_bm,plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm)

#función para derivar con respecto al tiempo a la función de Bono General
bonoMyieldm=function(x, plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm) #valoración bono tasa fija
{
  N=plazos_bm/plazocupon_bm+1   
  p1=plazos_bm-plazocupon_bm*(N-1)   
  a=(1-(1+x*plazocupon_bm/360)^(-N))/(plazocupon_bm*x/360)
  ((contratos_bm*nominal_bm*tfcupon_bm*plazocupon_bm/360)*a+(contratos_bm*nominal_bm)/((1+x*plazocupon_bm/360)^N))*(1+x*plazocupon_bm/360)^(1-p1/plazocupon_bm)   
}


mu_bm=eval(Deriv(bonoMyieldm(x0_bm,plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm), "plazos_bm"))
print(mu_bm)
g_bm=eval(Deriv(bonoMyield(x0_bm,plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm), "x0_bm"))
g_bmt=g_bm*x0_bm
gamma_bm=eval(Deriv(bonoMyield(x0_bm,plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm), "x0_bm",nderiv=2))
gamma_bmt2=diagv(gamma_bm*(x0_bm)^2)
print(gamma_bmt2)




            [,1] [,2]
[1,] -0.00670411  NaN
         [,1] [,2]
[1,] 30.54395    0
[2,]  0.00000   NA


## Bondes D

In [ ]:
%%R
#BONDE D CÁLCULO

X2_pr=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X2_orig_bd, on = .(Date),nomatch=0][order(-Date)] #Se alinea la tasa de fondeo por fecha con la tasa de valor presente.
m=length(plazos_bdm)


N_bd=as.integer(plazos_bdm/plazocupon_bdm)+1 #número de cupones a pagar
VTplazos_bdm=matrix(0,1,sum(N_bd)) #vector de todos los plazos_bdm de todos los contratos_bdm
contratos_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los contratos_bdm de todos los flujos de todos los contratos_bdm
nominal_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los nominal_bdmes de todos los flujos de todos los contratos_bdm
plazocupon_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los plazos_bdmcupon de todos los flujos de todos los contratos_bdm
tasafijaT_bd=matrix(0,1,sum(N_bd)) #vector de tasas fijas de todos los flujos de todos los contratos_bdm
ulNomT_bd=matrix(0,1,sum(N_bd)) #vector de contratos_bdm a final de flujo


plazini_bd=plazos_bdm-plazocupon_bdm*(N_bd-1) #vector de plazos_bdm iniciales
ddv=plazocupon_bdm-plazini_bd #dias trasncurridos del cupón vigente
tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
tfcupondev=matrix(0,1,m) #cupón de los días devengados
tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
#calcula cupones de bonos
for (j in (1:m))
{
  tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
  tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
}


for (j in (1:m))
{
  if (j==1)
  {
    VTplazos_bdm[,1:sum(N_bd[1:j])]=seq(plazini_bd[j],plazos_bdm[j], by=plazocupon_bdm[j])
    contratos_bdmT[,1:sum(N_bd[1:j])]=seq(contratos_bdm[j],contratos_bdm[j])
    plazocupon_bdmT[,1:sum(N_bd[1:j])]=seq(plazocupon_bdm[j],plazocupon_bdm[j])
    ulNomT_bd[,sum(N_bd[1:j])]=contratos_bdm[j]
    tasafijaT_bd[,1]=tfcupon[j]
    tasafijaT_bd[,2:sum(N_bd[1:j])]=seq(tfcupgen,tfcupgen)
  }
  else
  {
    VTplazos_bdm[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(plazini_bd[j],plazos_bdm[j], by=plazocupon_bdm[j])
    contratos_bdmT[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(contratos_bdm[j],contratos_bdm[j])					
    plazocupon_bdmT[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(plazocupon_bdm[j],plazocupon_bdm[j])
    tasafijaT_bd[,(sum(N_bd[1:j-1])+1)]=tfcupon[j]
    tasafijaT_bd[,(sum(N_bd[1:j-1])+2):sum(N_bd[1:j])]=seq(tfcupgen,tfcupgen)
    ulNomT_bd[,sum(N_bd[1:j])]=contratos_bdm[j]
  }
}

Xvp_bd=matrix(0,n,ncol(VTplazos_bdm))
Xst_bd=matrix(0,n,ncol(VTplazos_bdm))

for (i in (1:n))
{
  Xvp_bd[i,]=if(itpl==0){approx(nodos_gov,x_orig_gov[i,],VTplazos_bdm,rule=2)$y}else{talamb(nodos_gov,x_orig_gov[i,],VTplazos_bdm)}
  Xst_bd[i,]=if(itpl==0){approx(nodos3_bd,X3_orig_bd[i,],VTplazos_bdm,rule=2)$y}else{talamb(nodos3_bd,X3_orig_bd[i,],VTplazos_bdm)}
  
}


bondeD=function(contratos_bdmT,  nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp, Xst, N,ddv)
{
  tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
  tfcupondev=matrix(0,1,m) #cupón de los días devengados
  tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
  tasafijaT=matrix(0,1,sum(N))
  #calcula cupones de bonos
  for (j in (1:m))
  {
    tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
    tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
  }
  
  
  for (j in (1:m))
  {
    if (j==1)
    {
      tasafijaT[,1]=tfcupon[j]
      tasafijaT[,2:sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
    else
    {
      tasafijaT[,(sum(N[1:j-1])+1)]=tfcupon[j]
      tasafijaT[,(sum(N[1:j-1])+2):sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
  }
  
  
  V0=matrix(0,1,count(N))
  V0f=((((contratos_bdmT*(tasafijaT)*(plazocupon_bdmT/360))+ulNomT_bd)/(1+(Xvp+Xst)*VTplazos_bdm/360)))*nominal_bdm
  for (j in (1:count(N)))
  {
    if(j==1)
    {
      V0[j]=sum(V0f[j:N[j]])
    }
    else
    {
      V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
    }
  }
  V0
} 


V0_bdm=bondeD(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv)



d_bondeD_t=function(contratos_bdmT,  nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp, Xst, N,ddv)
{
  tasafijaT=matrix(0,1,sum(N))
  tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
  tfcupondev=matrix(0,1,m) #cupón de los días devengados
  tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
  #calcula cupones de bonos
  for (j in (1:m))
  {
    tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
    tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
  }
  
  
  for (j in (1:m))
  {
    if (j==1)
    {
      tasafijaT[,1]=tfcupon[j]
      tasafijaT[,2:sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
    else
    {
      tasafijaT[,(sum(N[1:j-1])+1)]=tfcupon[j]
      tasafijaT[,(sum(N[1:j-1])+2):sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
  }
  
  
  V0=matrix(0,1,count(N))
  V0f=-((((contratos_bdmT*(tasafijaT)*(plazocupon_bdmT/360))+ulNomT_bd)/(1+(Xvp+Xst)*VTplazos_bdm/360)^2))*nominal_bdm*((Xvp+Xst)/360)
  for (j in (1:count(N)))
  {
    if(j==1)
    {
      V0[j]=sum(V0f[j:N[j]])
    }
    else
    {
      V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
    }
  }
  V0
} 


#la derivada con respecto al valor presente y a la sobretasa es igual
d_bondeD_x=function(contratos_bdmT,  nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp, Xst, N,ddv)
{
  tasafijaT=matrix(0,1,sum(N))
  tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
  tfcupondev=matrix(0,1,m) #cupón de los días devengados
  tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
  #calcula cupones de bonos
  for (j in (1:m))
  {
    tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
    tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
  }
  
  
  for (j in (1:m))
  {
    if (j==1)
    {
      tasafijaT[,1]=tfcupon[j]
      tasafijaT[,2:sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
    else
    {
      tasafijaT[,(sum(N[1:j-1])+1)]=tfcupon[j]
      tasafijaT[,(sum(N[1:j-1])+2):sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
  }
  V0f=-((((contratos_bdmT*(tasafijaT)*(plazocupon_bdmT/360))+ulNomT_bd)/(1+(Xvp+Xst)*VTplazos_bdm/360)^2))*nominal_bdm*((VTplazos_bdm)/360)
  V0f
} 


d_bondeD_tc=function(contratos_bdmT,  nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp, Xst, N,ddv)
{
  tasafijaT=matrix(0,1,sum(N))
  tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
  tfcupondev=matrix(0,1,m) #cupón de los días devengados
  tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
  #calcula cupones de bonos
  for (j in (1:m))
  {
    tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
    tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
  }
  
  
  for (j in (1:m))
  {
    if (j==1)
    {
      tasafijaT[,1]=tfcupon[j]
      tasafijaT[,2:sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
    else
    {
      tasafijaT[,(sum(N[1:j-1])+1)]=tfcupon[j]
      tasafijaT[,(sum(N[1:j-1])+2):sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
  }
  
  V0f=-((((contratos_bdmT*(plazocupon_bdmT/360)))/(1+(Xvp+Xst)*VTplazos_bdm/360)))*nominal_bdm
  V0f
}



mgamma_bondeD=function(contratos_bdmT,  nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp, Xst, N,ddv)	#matriz de segundas derivadas parciales de la tasa valor presente y pago de flujos
{
  tasafijaT=matrix(0,1,sum(N))
  tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
  tfcupondev=matrix(0,1,m) #cupón de los días devengados
  tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
  #calcula cupones de bonos
  for (j in (1:m))
  {
    tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
    tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
  }
  
  
  for (j in (1:m))
  {
    if (j==1)
    {
      tasafijaT[,1]=tfcupon[j]
      tasafijaT[,2:sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
    else
    {
      tasafijaT[,(sum(N[1:j-1])+1)]=tfcupon[j]
      tasafijaT[,(sum(N[1:j-1])+2):sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
  }

  c11=matrix(0,1,length(contratos_bdmT)) #segunda derivada del cupón
  c12=-((((contratos_bdmT*(plazocupon_bdmT/360)))/(1+(Xvp+Xst)*VTplazos_bdm/360)^2))*nominal_bdm*((VTplazos_bdm)/360) #primera del cupón y primera del vp
  c13=c12 #primera del cupón y primera del st
  c21=-((((contratos_bdmT*(plazocupon_bdmT/360)))/(1+(Xvp+Xst)*VTplazos_bdm/360)^2))*nominal_bdm*((VTplazos_bdm)/360) #primera del vp y primera del cupon
  c22=2*((((contratos_bdmT*(tasafijaT)*(plazocupon_bdmT/360))+ulNomT_bd)/(1+(Xvp+Xst)*VTplazos_bdm/360)^3))*nominal_bdm*((VTplazos_bdm)/360)^2 #segunda del vp
  c23=c22 #primera del vp y primera del st
  c31=c12 #primera del st y primera del cupon
  c32=c22 #primera del vp y primera del st
  c33=c22 #segunda del st
  rbind(cbind(diagv(c11),diagv(c12), diagv(c13)),cbind(diagv(c21),diagv(c22), diagv(c23)),cbind(diagv(c31),diagv(c32), diagv(c33)))
}


#mgamma_bondeD(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv)

X_bd_tc=matrix(1,n,ncol(contratos_bdmT))*X2_pr$V2/100
X_bd_ext=cbind(X_bd_tc,as.matrix(Xvp_bd),as.matrix(Xst_bd))

mu_bd=d_bondeD_t(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv)
g_bd=cbind(d_bondeD_tc(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv),d_bondeD_x(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv),d_bondeD_x(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv))
x0_bd_ext=cbind(matrix(tf_act,1,ncol(contratos_bdmT)),t(as.matrix(Xvp_bd[1,])),t(as.matrix(Xst_bd[1,])))
g_bdt=g_bd*x0_bd_ext
gamma_bd=mgamma_bondeD(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv)
gamma_bdt2=gamma_bd*(t(x0_bd_ext)%*%x0_bd_ext)


## Forwards Tipo de Cambio

In [ ]:
%%R
#FORWARDS Y/O FUTUROS DE TIPO DE CAMBIO CÁLCULO

################MATRICES DE INTERPOLACION LINEAL ####################

m=ncol(plazos_fwd)
X1_fwtdc=matrix(0,n,m)
X2_fwtdc=matrix(0,n,m)

for (j in 1:n)
{
  X1_fwtdc[j,]=if(itpl==0){approx(nodos1_ftdc,x1_ftdc[j,],plazos_fwd,rule=2)$y}else{talamb(nodos1_ftdc,x1_ftdc[j,],plazos_fwd)}
  X2_fwtdc[j,]=if(itpl==0){approx(nodos2_ftdc,x2_ftdc[j,],plazos_fwd,rule=2)$y}else{talamb(nodos2_ftdc,x2_ftdc[j,],plazos_fwd)}
  if(trlib==1){X1_fwtdc[j,]=((1+X1_fwtdc[j,])^(plazos_fwd/180)-1)*360/plazos_fwd} #transformación de act 180 a 360
}


futuroTC = function(t,tl,tn,s,k) #t=dias por vencer, tn=tasa nacional para tipo de cambio forward, tl= tasa extranjera pra tipo de cambio forward, S=spot
{
  f=s*((1+tn*t/360)/(1+tl*t/360)) #Se obtiene el tipo de cambio forward
  t(as.numeric((f-k)/(1+t*tn/360))) #Se obtiene el valor del payoff a valor presente con el valor z que define si es largo o corto
}

X3_ftdc=as.matrix(X3_ftdc)

V0_fwtdc=futuroTC(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd)*contratos_fwd*nominal_fwd

#########################
dfuturoTC_tl = function(t,tl,tn,s,k) #t=dias por vencer, tn=tasa nacional para tipo de cambio forward, tl= tasa extranjera pra tipo de cambio forward, S=spot
{
  f=-s*((1+tn*t/360)/(1+tl*t/360)^2)*(t/360) #Se obtiene el tipo de cambio forward
  t(as.numeric((f)/(1+t*tn/360))) #Se obtiene el valor del payoff a valor presente con el valor z que define si es largo o corto
}

dfuturoTC_tn = function(t,tl,tn,s,k) #t=dias por vencer, tn=tasa nacional para tipo de cambio forward, tl= tasa extranjera pra tipo de cambio forward, S=spot
{
  f=s*((t/360)/(1+tl*t/360)) #Se obtiene el tipo de cambio forward
  f1=s*((1+tn*t/360)/(1+tl*t/360)) #Se obtiene el tipo de cambio forward
  t(as.numeric((f)/(1+t*tn/360)-(f1-k)/(1+t*tn/360)^2*(t/360))) #Se obtiene el valor del payoff a valor presente con el valor z que define si es largo o corto
}

dfuturoTC_s = function(t,tl,tn,s,k) #t=dias por vencer, tn=tasa nacional para tipo de cambio forward, tl= tasa extranjera pra tipo de cambio forward, S=spot
{
  1/(1+t*tl/360) #Se obtiene el valor del payoff a valor presente con el valor z que define si es largo o corto
}

dfuturoTC_t = function(t,tl,tn,s,k) #t=dias por vencer, tn=tasa nacional para tipo de cambio forward, tl= tasa extranjera pra tipo de cambio forward, S=spot
{
  f=s*((1+tn/360)/(1+tl*t/360))-s*((1+tn*t/360)/(1+tl*t/360)^2)*(tl/360) #Se obtiene el tipo de cambio forward
  #t(as.numeric((f)/(1+t*tn/360)-(f-k)/(1+t*tn/360)^2*(tn/360))) #Se obtiene el valor del payoff a valor presente con el valor z que define si es largo o corto
  -(f-k)/(1+t*tn/360)^2*(tn/360)
}

mgamma_futTC= function(t,tl,tn,s,k,contratos_fwd,nominal_fwd)
{
c11= matrix(0,1,length(k))#spot spot
c12= -(((1+tn*t/360)/(1+tl*t/360)^2)*(t/360))/(1+t*tn/360)*contratos_fwd*nominal_fwd#spot tl
c13=  matrix(0,1,length(k))  #spot tn
c21=  -(((1+tn*t/360)/(1+tl*t/360)^2)*(t/360))/(1+t*tn/360)*contratos_fwd*nominal_fwd #tl spot
c22= (2*s*((1+tn*t/360)/(1+tl*t/360)^3)*(t/360)^3)/(1+t*tn/360)*contratos_fwd*nominal_fwd #tl tl
c23= matrix(0,1,length(k)) #tl tn
c31=(((t/360)/(1+tl*t/360))/(1+t*tn/360)-(((1+tn*t/360)/(1+tl*t/360)))/(1+t*tn/360)^2*(t/360))*contratos_fwd*nominal_fwd #tn spot
c32=   (-s*((t/360)^2/(1+tl*t/360)^2)/(1+t*tn/360)-(s*((t/360)/(1+tl*t/360)))/(1+t*tn/360)^2*(t/360))*contratos_fwd*nominal_fwd #tn tl
c33=  (s*((t/360)/(1+tl*t/360))/(1+t*tn/360)^2*(t/360)+2*(s*((1+tn*t/360)/(1+tl*t/360))-k)/(1+t*tn/360)^3*(t/360)^2-(s*((t/360)^2/(1+tl*t/360)))/(1+t*tn/360)^2)*contratos_fwd*nominal_fwd #tn tn
m=length(t)
rbind(cbind(diagv(c11),diagv(c12),diagv(c13)),cbind(diagv(c21),diagv(c22),diagv(c23)),cbind(diagv(c31),diagv(c32),diagv(c33)))
}


X_ftdc=cbind(matrix(X3_ftdc,n,ncol(X1_fwtdc)),X1_fwtdc,X2_fwtdc) #Hist facto riesgo
xp_ftdc=t(as.matrix(X_ftdc[1,])) #Valores actuales X0
mu_ftdc=dfuturoTC_t(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd)*contratos_fwd*nominal_fwd #primera derivada con respecto al tiempo evaluada en x0
g_ftdc=cbind(dfuturoTC_s(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd)*contratos_fwd*nominal_fwd,dfuturoTC_tl(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd)*contratos_fwd*nominal_fwd,dfuturoTC_tn(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd)*contratos_fwd*nominal_fwd) #gradiente evaluado en x0
g_ftdctT=xp_ftdc*g_ftdc #gradienete evaluado en x0 por x0
gamma_ftdc=mgamma_futTC(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd,contratos_fwd,nominal_fwd) #evaluación de gamma
gamma_ftdctT2=gamma_ftdc*(t(xp_ftdc)%*%xp_ftdc) #evaluación de gamma multiplicado por x^{2}

print(gamma_ftdc)
print(gamma_ftdc)



              [,1]        [,2]        [,3]
[1,]  0.000000e+00 -1.38739259  0.00000000
[2,] -1.387393e+00  0.01031777  0.00000000
[3,]  3.469447e-16 -0.74227440 -0.02049339
              [,1]        [,2]        [,3]
[1,]  0.000000e+00 -1.38739259  0.00000000
[2,] -1.387393e+00  0.01031777  0.00000000
[3,]  3.469447e-16 -0.74227440 -0.02049339


## Forwards Acciones o Índices

In [ ]:
%%R
#FORWARDS Y/O FUTUROS DE ÍNDICES CÁLCULO

################MATRICES DE INTERPOLACION LINEAL ####################
m=ncol(plazos_fwd_ind)
X1_fwind=matrix(0,n,m) #DIVIDENDOS
X2_fwind=matrix(0,n,m)
 
for (j in 1:n){
  #X1_fwind[j,]=if(itpl==0){approx(nodos2_div,x2_fdiv[j,],plazos_fwd_ind, rule=2)$y}else{talamb(nodos2_div,x2_fdiv[j,],plazos_fwd_ind)}
  X2_fwind[j,]=if(itpl==0){approx(nodos_gov,x_orig_gov[j,],plazos_fwd_ind,rule=2)$y}else{talamb(nodos_gov,x_orig_gov[j,],plazos_fwd_ind)}
}

futuroI = function(t,td,tn,s,k){ #t=dias por vencer, tn=tasa nacional para indice forward, td= tasa dividendos forward, S=spot
  f=s*(1+(tn-td)*t/360)          #Se obtiene el tipo de cambio forward
  t(as.numeric((f-k)/(1+t*tn/360))) #Se obtiene el valor del payoff a valor presente con el valor z que define si es largo o corto
}

dfuturoI_tl = function(pv,td,tl,s,k){#pv= plazo de vencimiento de la operacion, td = Tasa de dividendos a pv dias, tl = tasa libre de riesgo
                                   #s= valor a la fecha de valuacion del indice, K = Strike
    f = ((s*td*(pv^2))/((360^2)*(1+(tl*pv/360)^2))) + (k*pv)/(360*(1+(tl*pv/360)^2))
    f
  }
dfuturoI_td = function(pv,td,tl,s,k){#pv= plazo de vencimiento de la operacion, td = Tasa de dividendos a pv dias, tl = tasa libre de riesgo
                                   #s= valor a la fecha de valuacion del indice, K = Strike
    f = -(pv*s)/(360+tl*pv)
    f
}
dfuturoI_t = function(pv,td,tl,s,k){#pv= plazo de vencimiento de la operacion, td = Tasa de dividendos a pv dias, tl = tasa libre de riesgo
                                   #s= valor a la fecha de valuacion del indice, K = Strike
    f = ((-s*td)/(360))*(((1+(tl*pv)/360)-((tl*pv)/360))/((1+(tl*pv)/360)^2))+(k*tl)/(360*(1+(tl*pv)/360)^2)     
    f    
}
dfuturoI_s = function(pv,td,tl,s,k){#pv= plazo de vencimiento de la operacion, td = Tasa de dividendos a pv dias, tl = tasa libre de riesgo
                                   #s= valor a la fecha de valuacion del indice, K = Strike
     f = (1+(tl-td)*(pv/360))/(1+tl*pv/360)                                                                                               
     f 
}

mgamma_futI= function(t,td,tn,s,k,contratos_fwd,nominal_fwd){
c11= matrix(0,1,length(k))#spot spot
c12= -1/(tn)*contratos_fwd*nominal_fwd#spot td
c13=  ((((1+tn*t/360)*(t/360))-((1+(tn-td)*(t/360)^2)))/(1+tn*t/360)^2)*contratos_fwd*nominal_fwd#spot tn
c21=  -((t/360)/(1+tn*t/360)) *contratos_fwd*nominal_fwd #td spot
c22= matrix(0,1,length(k)) #td td
c23= ((s*(t/360)^2)/(1+tn*t/360)^2)*contratos_fwd*nominal_fwd #td tn
c31=((((1+tn*t/360)*(t/360))-((1+(tn-td)*(t/360)^2)))/(1+tn*t/360)^2)*contratos_fwd*nominal_fwd #tn spot
c32=   ((s*(t/360)^2)/(1+tn*t/360)^2)*contratos_fwd*nominal_fwd #tn td
c33=   ((((1+tn*t/360)*(t/360))-((1+(tn-td)*(t/360)^2)))/(1+tn*t/360)^2)*(4*(1+tn*t/360)^3*t/360)/(1+tn*t/360)^2*contratos_fwd*nominal_fwd #tn tn
m=length(t)
rbind(cbind(diagv(c11),diagv(c12),diagv(c13)),cbind(diagv(c21),diagv(c22),diagv(c23)),cbind(diagv(c31),diagv(c32),diagv(c33)))
}

X3_find=as.matrix(X3_find)
#X_futind=cbind(X1_fwind,X2_fwind,matrix(X3_find,n,ncol(X1_fwind)))
 
V0_fwind=futuroI(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind
print(V0_fwind)

X_find=cbind(matrix(X3_find,n,ncol(X1_fwind)),X1_fwind,X2_fwind)
xp_find=t(as.matrix(X_find[1,]))
mu_find=dfuturoI_t(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind
g_find=cbind(dfuturoI_s(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind,dfuturoTC_tl(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind,dfuturoTC_tn(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind)
g_findtT=xp_find*g_find
gamma_find=mgamma_futI(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind,contratos_fwd_ind,nominal_fwd_ind)
gamma_findtT2=gamma_find*(t(xp_find)%*%xp_find)

print(gamma_find)




          [,1]
[1,] -367656.2
           [,1]       [,2]        [,3]
[1,]   0.000000  -638.4995   -41.67249
[2,]  -7.277214     0.0000 44068.78464
[3,] -41.672485 44068.7846   -24.82338


## SWAPS

In [ ]:
%%R
##SWAP TASA FIJA VS TASA VARIABLE CÁLCULO

##Interpolamos

nodosvp=nodos1_sw
nodostc=nodos2_sw
curvavp=as.matrix(X1_orig_sw)
curvatc=X2_orig_sw
n1=nrow(curvavp)
n2=nrow(curvatc)


m=max(ncol(plazos_sw),1) #número de contratos_sw swap a valorar

N=matrix(0,1,m) #es un vector de m valores donde se cargarán los m número de cupones a pagar para cada contrato

for (j in (1:m))
{
  N[j]=as.integer(plazos_sw[j]/plazocupon_sw[j])+1 #número de cupones a pagar
}

VTplazos_sw=matrix(0,1,sum(N)) #vector de todos los plazos_sw de todos los contratos_sw
contratos_swT=matrix(0,1,sum(N)) #vector de todos los contratos_sw de todos los flujos de todos los contratos_sw
nominal_swT=matrix(0,1,sum(N)) #vector de todos los nominal_swes de todos los flujos de todos los contratos_sw
por_swT=matrix(0,1,sum(N)) #vector de todos los dummy si paga o recibe de todos los flujos de todos los contratos_sw
plazocupon_swT=matrix(0,1,sum(N)) #vector de todos los plazos_swcupon de todos los flujos de todos los contratos_sw
tasafija_swT=matrix(0,1,sum(N)) #vector de tasas fijas de todos los flujos de todos los contratos_sw
VTplazos_swc=matrix(0,1,sum(N)) #vector de todos los plazos_sw cortos de todos los contratos_sw


plazini=plazos_sw-plazocupon_sw*(N-1) #vector de plazos_sw iniciales

for (j in (1:m))
{
  if (j==1)
  {
    VTplazos_sw[,1:sum(N[1:j])]=seq(plazini[j],plazos_sw[j], by=plazocupon_sw[j])
    VTplazos_swc[,1:sum(N[1:j])]=c(0,VTplazos_sw[,1:(sum(N[1:j])-1)])
    contratos_swT[,1:sum(N[1:j])]=seq(contratos_sw[j],contratos_sw[j])
    nominal_swT[,1:sum(N[1:j])]=seq(nominal_sw[j],nominal_sw[j])
    por_swT[,1:sum(N[1:j])]=seq(por_sw[j],por_sw[j])
    plazocupon_swT[,1:sum(N[1:j])]=seq(plazocupon_sw[j],plazocupon_sw[j])
    tasafija_swT[,1:sum(N[1:j])]=seq(tasafija_sw[j],tasafija_sw[j])
  }
  else
  {
    VTplazos_sw[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazini[j],plazos_sw[j], by=plazocupon_sw[j])
    VTplazos_swc[,(sum(N[1:j-1])+1):sum(N[1:j])]=c(0,VTplazos_sw[,(sum(N[1:j-1])+1):(sum(N[1:j])-1)])
    contratos_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(contratos_sw[j],contratos_sw[j])					
    nominal_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(nominal_sw[j],nominal_sw[j])
    por_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(por_sw[j],por_sw[j])
    plazocupon_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazocupon_sw[j],plazocupon_sw[j])
    tasafija_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(tasafija_sw[j],tasafija_sw[j])
  }
}


Xvp=matrix(0,n,ncol(VTplazos_sw))
Xtc=matrix(0,n,ncol(VTplazos_sw))
Xtcc=matrix(0,n,ncol(VTplazos_sw))
XtfwdT=matrix(0,n,ncol(VTplazos_sw))

for (i in (1:n))
{
  Xvp[i,]=if(itpl==0){approx(nodosvp,curvavp[i,],VTplazos_sw,rule=2)$y}else{talamb(nodosvp,curvavp[i,],VTplazos_sw)}
  Xtc[i,]=if(itpl==0){approx(nodostc,curvatc[i,],VTplazos_sw,rule=2)$y}else{talamb(nodostc,curvatc[i,],VTplazos_sw)}
  Xtcc[i,]=if(itpl==0){approx(nodostc,curvatc[i,],VTplazos_swc, rule=2)$y}else{talamb(nodostc,curvatc[i,],VTplazos_swc)}
  
  XtfwdT[i,]=((1+Xtc[i,]*VTplazos_sw/360)/(1+Xtcc[i,]*VTplazos_swc/360)-1)*360/plazocupon_swT 
  for (j in (1:ncol(VTplazos_sw)))
  {
    if (VTplazos_sw[j]<= plazocupon_swT[j])
    {
      XtfwdT[i,j]=Xtc[i,j]
    }
    else
    {
      j=sum(N[1:j])
    }
  }
}


swap=function(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp, N)
{
  V0=matrix(0,1,ncol(N))
  V0f=(((contratos_swT*(XtfwdT-tasafija_swT)*(plazocupon_swT/360)))/(1+Xvp*VTplazos_sw/360))*nominal_swT*(-1)^por_swT
  for (j in (1:ncol(N)))
  {
    if(j==1)
    {
      V0[j]=sum(V0f[j:N[j]])
    }
    else
    {
      V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
    }
  }
  V0
}

V0_sw=swap(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,], N)

#derivada del swap con respecto aL tiempo
dswap_dtt=function(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp)
{
  -1*(((contratos_swT*(XtfwdT-tasafija_swT)*(plazocupon_swT*Xvp/360^2)))/(1+Xvp*VTplazos_sw/360)^(2))*nominal_swT*(-1)^por_swT
}

#derivada del swap con respecto a la tasa cupón variable
dswap_dtc=function(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp)
{
  (((contratos_swT*plazocupon_swT)/360)/(1+Xvp*VTplazos_sw/360))*nominal_swT*(-1)^por_swT
}

#derivada del swap con respecto al valor presente
dswap_dvp=function(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp)
{
  -1*(((contratos_swT*(XtfwdT-tasafija_swT)*(plazocupon_swT*VTplazos_sw/360^2)))/(1+Xvp*VTplazos_sw/360)^(2))*nominal_swT*(-1)^por_swT
}

mgamma_swap=function(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp)	#matriz de segundas derivadas parciales de la tasa valor presente y pago de flujos
{
  c11=matrix(0,1,ncol(contratos_swT))
  c21=-1*(((contratos_swT*(plazocupon_swT*VTplazos_sw/360^2)))/(1+Xvp*VTplazos_sw/360)^(2))*nominal_swT*(-1)^por_swT
  c12=-1*(((contratos_swT*plazocupon_swT*VTplazos_sw)/360^2)/(1+Xvp*VTplazos_sw/360)^2)*nominal_swT*(-1)^por_swT
  c22=2*(((contratos_swT*(XtfwdT-tasafija_swT)*(plazocupon_swT*VTplazos_sw^2/360^3)))/(1+Xvp*VTplazos_sw/360)^(3))*nominal_swT*(-1)^por_swT
  rbind(cbind(diagv(c11),diagv(c12)),cbind(diagv(c21),diagv(c22)))
}

X_sw=cbind( XtfwdT,Xvp)

mu_sw=dswap_dtt(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,])
g_sw=cbind(dswap_dtc(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,]),dswap_dvp(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,]))

#Multiplicación de la derivada por el punto x0
gt_sw=c( XtfwdT[1,],Xvp[1,])*g_sw

gamma_sw=mgamma_swap(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,])

tgamma_sw=gamma_sw*c( XtfwdT[1,],Xvp[1,])%*%t(c( XtfwdT[1,],Xvp[1,]))



##Opciones de Tasa de Interés

In [ ]:
%%R


##opciones de tasa de interés, con inicio el día de la valuación CÁLCULO
#Posición inicial

#interpolación de tasas y volatilidades
m=ncol(plazos_oir)
x1=matrix(0,n,m)
x2tc=matrix(0,n,m)
x2tl=matrix(0,n,m)
x2=matrix(0,n,m)
x3=matrix(0,n,m)
for (i in 1:(n))
{
  x1[i,]=if(itpl==0){approx(nodos1_oir,x1_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos1_oir,x1_orig_oir[i,],plazos_oir)}
  x2tc[i,]=if(itpl==0){approx(nodos2_oir,x2_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos2_oir,x2_orig_oir[i,],plazos_oir)}
  x2tl[i,]=if(itpl==0){approx(nodos2_oir,x2_orig_oir[i,],(plazos_oir+pr_oir),rule=2)$y}else{talamb(nodos2_oir,x2_orig_oir[i,],(plazos_oir+pr_oir))}
  x3[i,]=if(itpl==0){approx(nodos3_oir,x3_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos3_oir,x3_orig_oir[i,],plazos_oir)}
  x2[i,]=((1+x2tl[i,]*(plazos_oir+pr_oir)/360)/(1+x2tc[i,]*(plazos_oir)/360)-1)*360/pr_oir
}


x01=x1[1,] #tasas de descuento
x02=x2[1,] #tasas spot
x03=x3[1,] #volatilidades



opctint = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#función de una opción europea
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}  
  (if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir}else{(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360))*(-1)^cp_oir})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)
}


V0_oir=opctint(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio


#pr_oirIMERA DERIVADA

dopctintt = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#derivada parcial europea y TIEMPO
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}  
  (if(cs_oir==1){(( -(S*dnorm(d1)*vol)/(2*sqrt(t/365))+(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(t/365)*(-1)^cp_oir)*(exp(-vp*t/365)))}else{(( -(S*dnorm(d1)*vol)/(2*sqrt(t/360))+(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(t/360)*(-1)^cp_oir)*(1/(1+vp*t/360)^2))})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)
}

#dopctintt(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)


dopctintS = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#derivada parcial call europea y spot
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}  
  (if(cs_oir==1){((pnorm(d1*(-1)^cp_oir))*(exp(-vp*t/365)))}else{(pnorm(d1*(-1)^cp_oir)*(1/(1+vp*t/360)))})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)- (if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir}else{(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360))*(-1)^cp_oir})*(pr_oir/dct_oir)^2/(1+S*pr_oir/dct_oir)^2
}

#dopctintS(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)

dopctintvol = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#derivada parcial call europea y spot
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}    
  (if(cs_oir==1){(S*dnorm(d1)*(sqrt(t/365)))*(exp(-vp*t/365))}else{(S*dnorm(d1)*(sqrt(t/360)))*(1/(1+vp*t/360))})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)
}

#dopctintvol(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)

dopctintrho = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#derivada parcial call europea y rho
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}    
  (if(cs_oir==1){-(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir*(t/365)}else{-(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360)^2)*(-1)^cp_oir*(t/360)})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)
}

#dopctintrho(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)


mgammaopc=function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir,contratos_oir,nominal_oir)	#matriz de segundas derivadas parciales de la call: rho, spot y vol de spot
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}    
  c11=(if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir*(t/365)^2}else{2*(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360)^3)*(-1)^cp_oir*(t/360)^2})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)*contratos_oir*nominal_oir #rho con respecto a rho
  c21=(if(cs_oir==1){((pnorm(d1*(-1)^cp_oir))*(exp(-vp*t/365)))*(-t/365)}else{(pnorm(d1*(-1)^cp_oir)*(1/(1+vp*t/360)^2))*(-t/360)})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)- (if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir*(-t/365)}else{(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360)^2)*(-1)^cp_oir*(-t/360)})*(pr_oir/dct_oir)^2/(1+S*pr_oir/dct_oir)^2*contratos_oir*nominal_oir #spot con respecto a rho
  c31=(if(cs_oir==1){(S*dnorm(d1)*(sqrt(t/365)))*(exp(-vp*t/365))*(-t/365)}else{(S*dnorm(d1)*(sqrt(t/360)))*(1/(1+vp*t/360)^2)*(-t/360)})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)*contratos_oir*nominal_oir #vol con respecto a rho
  c12=c21 #rho con respecto a spot
  c22=(if(cs_oir==1){(dnorm(d1)/(S*vol*sqrt(t/365)))*(exp(-vp*t/365))}else{(dnorm(d1)/(S*vol*sqrt(t/360)))*(1/(1+vp*t/360))})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir) -2*((if(cs_oir==1){(((pnorm(d1*(-1)^cp_oir))+( S*dnorm(d1*(-1)^cp_oir)-K_oir*dnorm(d2*(-1)^cp_oir)))*(exp(-vp*t/365)))*(-1)^cp_oir*(1/(S*vol*sqrt(t/365)))}else{(((pnorm(d1*(-1)^cp_oir))+( S*dnorm(d1*(-1)^cp_oir)-K_oir*dnorm(d2*(-1)^cp_oir)))*(1/(1+vp*t/360)))*(1/(S*vol*sqrt(t/360)))*(-1)^cp_oir})*(pr_oir/dct_oir)^2/(1+S*pr_oir/dct_oir)^2) +2*(if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir}else{(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360))*(-1)^cp_oir})*(pr_oir/dct_oir)^3/(1+S*pr_oir/dct_oir)^3*contratos_oir*nominal_oir #spot spot 
  c32=(if(cs_oir==1){dnorm(d1)*d2/vol*(exp(-vp*t/365))}else{dnorm(d1)*d2/vol*(1/(1+vp*t/360))})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir) +   (if(cs_oir==1){(S*dnorm(d1)*(sqrt(t/365)))*(exp(-vp*t/365))}else{(S*dnorm(d1)*(sqrt(t/360)))*(1/(1+vp*t/360))})*(pr_oir/dct_oir)^2/(1+S*pr_oir/dct_oir)^2*contratos_oir*nominal_oir # vol spot
  c13=(if(cs_oir==1){(S*dnorm(d1)*(sqrt(t/365)))*(exp(-vp*t/365))*(-t/365)}else{(S*dnorm(d1)*(sqrt(t/360)))*(1/(1+vp*t/360)^2)*(-t/360)})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)*contratos_oir*nominal_oir #rho vol
  c23=c32 #spot vol
  c33=(if(cs_oir==1){(dnorm(d1)*(S*sqrt(t/365)))*((d1*d2)/vol)*exp(-vp*t/365)}else{(dnorm(d1)*(S*sqrt(t/360)))*((d1*d2)/vol)*(1/(1+vp*t/360))})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)*contratos_oir*nominal_oir #vol vol
  # outm=matrix(0,3,length(plazos_oir)*3)
  m=length(t)
 rbind(cbind(diagv(c11),diagv(c12),diagv(c13)),cbind(diagv(c21),diagv(c22),diagv(c23)),cbind(diagv(c31),diagv(c32),diagv(c33)))
}

#mgammaopc(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir,contratos_oir,nominal_oir)
X_oir=cbind(x1,x2,x3)
xp_oir=cbind(t(x01),t(x02),t(x03))
mu_oir=dopctintt(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir 
delta_oir_T=cbind(dopctintrho(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir, dopctintS(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir,dopctintvol(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir)
deltaT_oir_T=xp_oir*delta_oir_T
gammaT_oir=mgammaopc(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir,contratos_oir,nominal_oir)
gammaT_oirT2=gammaT_oir*(t(xp_oir)%*%xp_oir)



# Integración de factores y cálculo de riesgo en conjunto

In [ ]:
%%R
#DIMENSION DE TODOS LOS INSTRUMENTOS
#Son 8 instrumentos financieros (9 si separamos acciones y divisas)
n_if=matrix(0,8,1)
n_if[1]=ncol(stock_prices_EQFX) #acciones y divisas
n_if[2]=ncol(x_bcc) #cetes
n_if[3]=ncol(X_bm) #bonos m
n_if[4]=ncol(X_bd_ext) #bonde
n_if[5]=ncol(X_ftdc) #fut tdc
n_if[6]=ncol(X_find) #fut ind
n_if[7]=ncol(X_sw) #swaps
n_if[8]=ncol(X_oir) #opciones tasa de interés

#valor del portafolios

V0_port=cbind(t(as.matrix(V0_bcc)), V0_bm, V0_bdm, V0_fwtdc, V0_fwind, V0_sw, V0_oir)
V0T_port=sum(V0_port)



#INTEGRACIÓN DE TODOS LOS FACTORES DE RIESGO EN UNA MATRIZ
X_port=cbind(stock_prices_EQFX,x_bcc,X_bm,X_bd_ext,X_ftdc,X_find,X_sw,X_oir) #Factores de riesgo del portafolios de 8(9) instrumentos financieros

#Cálculo de variaciones Delta_X DEL PORTAFOLIOS
DeltaX_port=as.matrix(log(X_port[1:(n-1)]/X_port[2:(n)]))
DeltaX_port[is.nan(DeltaX_port)] <- 0 #quitamos NaN
DeltaX_port[is.na(DeltaX_port)] <- 0 #quitamos Na
DeltaX_port[is.infinite(DeltaX_port)] <- 0 #quitamos Na


# Medición de Riesgo

In [ ]:
%%R

for (i in 1:(m)){
  x0_acc_div<-as.matrix(x0_acc_div)
  delta_acc_div=t(g_acc_div[i]) 
  #print(delta_acc_div)
  tdelta_acc_div=t(g_acc_div[i])*x0_acc_div[i]   #delta tilde
  gamma_acc_div=gamma_acc_divt[i]##########
  tgamma_acc_div=gamma_acc_div*(x0_acc_div[i])^2 #gamma tilde
  Varmargcaux_acc_div=var(DeltaX_acc_div[,i])*((n)/(n-1)) #matriz varcov factores de riesgo de cada contrato
  
  
  tgammavar_acc_div=(tgamma_acc_div %*% Varmargcaux_acc_div)
  #print(tgamma_acc_div)
  #print(Varmargcaux_acc_div)
  print(tgamma_acc_div)
}


[1] 0
[1] 0


##Acciones y divisas 

### Riesgo de contrato y nivel contrato-factor de riesgo

In [ ]:
%%R
#Cálculo de VaR por contrato
#creación de matrices de VaR, gamma, VaRCF por contrato
DeltaX_acc_div=DeltaX_port[,1:(sum(n_if[1]))] 
m = length(pos_eq) + length(pos_fx)
m_a<-length(pos_eq)
m_d<-length(pos_fx)
VaRmargc_acc_div=matrix(0,1,m) #Var por contrato Delta GammaNormal
VaRmargcdn_acc_div=matrix(0,1,m) #Var por Delta Normal
VaRmargcstd_acc_div=matrix(0,1,m) #Var por contrato estandarizado 
VaRmargcCF_acc_div=matrix(0,1,m) #Var por contrato Cornish-Fisher
CVaRmargcdn_acc_div=matrix(0,1,m) 
CVaRmargc_acc_div=matrix(0,1,m) #CVar por contrato estandarizado
CVaRmargcCF_acc_div=matrix(0,1,m) #CVar por contrato estandarizado

##factores riesgo
#Acciones
VaRmargc_fr_acc=matrix(0,1,m_a) #Var por contrato Delta Normal
VaRmargcstd_fr_acc=matrix(0,1,m_a) #Var por contrato estandarizado 
VaRmargcCF_fr_acc=matrix(0,1,m_a) #Var por contrato Cornish-Fisher
CVaRmargc_fr_acc=matrix(0,1,m_a) #CVar por contrato estandarizado
CVaRmargcCF_fr_acc=matrix(0,1,m_a) #CVar por contrato estandarizado
#Divisas 
VaRmargc_fr_div=matrix(0,1,m_d) #Var por contrato Delta Normal
VaRmargcstd_fr_div=matrix(0,1,m_d) #Var por contrato estandarizado 
VaRmargcCF_fr_div=matrix(0,1,m_d) #Var por contrato Cornish-Fisher
CVaRmargc_fr_div=matrix(0,1,m_d) #CVar por contrato estandarizado
CVaRmargcCF_fr_div=matrix(0,1,m_d) #CVar por contrato estandarizado

WFR=diagv(c(1))

mu_acc_div=matrix(0,1,ncol(x0_acc_div))
g_acc_div=cbind(t(pos_fx),t(pos_eq))
g_acc_divt=cbind(t(pos_fx),t(pos_eq))*x0_acc_div
gamma_acc_divt=matrix(0,ncol(x0_acc_div),ncol(x0_acc_div))

for (i in 1:(m)){
  x0_acc_div<-as.matrix(x0_acc_div)
  delta_acc_div=t(g_acc_div[i]) 
  tdelta_acc_div=t(g_acc_div[i])*x0_acc_div[i]   #delta tilde
  gamma_acc_div=gamma_acc_divt[i]##########
  tgamma_acc_div=gamma_acc_div*(x0_acc_div[i])^2 #gamma tilde
  Varmargcaux_acc_div=var(DeltaX_acc_div[,i])*((n)/(n-1)) #matriz varcov factores de riesgo de cada contrato
  #print(Varmargcaux_acc_div)
  tgammavar_acc_div=(tgamma_acc_div %*% Varmargcaux_acc_div)

  tgammavar2_acc_div=(tgamma_acc_div %*% Varmargcaux_acc_div)%*%(tgamma_acc_div %*% Varmargcaux_acc_div)  
  mudg_acc_div=ifelse((sum(diag(tgammavar_acc_div))/2+mu_acc_div[i])>0,0,(sum(diag(tgammavar_acc_div))/2+mu_acc_div[i]))
  desv_acc_div=sqrt(sum(diag(tgammavar2_acc_div))/2+tdelta_acc_div %*% Varmargcaux_acc_div %*% t(tdelta_acc_div)) #esta es la unica uqe no se hace cero 

  VaRmargc_acc_div[i]=mudg_acc_div+desv_acc_div*qnorm(1-alpha) #VaR Normal por contrato
  CVaRmargc_acc_div[i]=-mudg_acc_div-desv_acc_div*(dnorm(qnorm(1-alpha))/(1-alpha))

  #Delta normal
  mudgdn_acc_div=ifelse((mu_acc_div[i])>0,0,mu_acc_div[i]) 
  desvdn_acc_div=sqrt(tdelta_acc_div %*% Varmargcaux_acc_div %*% t(tdelta_acc_div))

  VaRmargcdn_acc_div[i]=mudgdn_acc_div+desvdn_acc_div*qnorm(1-alpha) #VaR Normal por contrato
  CVaRmargcdn_acc_div[i]=-mudgdn_acc_div-desvdn_acc_div*(dnorm(qnorm(1-alpha))/(1-alpha))
  
  tgammavar3_acc_div=tgammavar2_acc_div %*% (tgamma_acc_div %*% Varmargcaux_acc_div)
  asimPmargc_acc_div=sum(diag(tgammavar3_acc_div))+3*tdelta_acc_div %*% Varmargcaux_acc_div %*% tgamma_acc_div %*% Varmargcaux_acc_div %*% t(tdelta_acc_div) #tercer momento
  asimPmargc_acc_div=asimPmargc_acc_div/(desv_acc_div)^3  #coef de asimetría
  VaRmargcstd_acc_div[i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_acc_div  #VaR estandarizado
  VaRmargcCF_acc_div[i]=mudg_acc_div+desv_acc_div*VaRmargcstd_acc_div[i] 	    #VaR Cornish-Fisher
  CVaRmargcCF_acc_div[i]=-mudg_acc_div-desv_acc_div*(dnorm(VaRmargcstd_acc_div[i])/(pnorm(VaRmargcstd_acc_div[i])))
#Por factor de riesgo y por contrato
#spot, tasa extranjera, tasa nacional
#Dado que solo se tiene un factor de riesgo eliminamos el for
}

print("Valor a nivel contrato")
print(V0_acc_div)
print("VaR a nivel contrato DN")
print(VaRmargcdn_acc_div)
print("CVaR a nivel contrato DN")
print(CVaRmargcdn_acc_div)
print("VaR a nivel contrato")
print(VaRmargc_acc_div)
print("VaR a nivel contrato CF")
print(VaRmargcCF_acc_div)
print("CVaR a nivel contrato")
print(CVaRmargc_acc_div)
print("CVaR a nivel contrato CF")
print(CVaRmargcCF_acc_div)


[1] "Valor a nivel contrato"
   EURUSD.X GBPUSD.X USDMXN.X AMXL.MX GCARSOA1.MX WALMEX.MX
1: 14696.63 -14935.8  28929.9  -77350       56720     66408
[1] "VaR a nivel contrato DN"
          [,1]      [,2]      [,3]      [,4]      [,5]      [,6]
[1,] -145.8201 -156.1109 -240.6571 -2018.004 -2371.261 -1736.707
[1] "CVaR a nivel contrato DN"
          [,1]      [,2]      [,3]      [,4]      [,5]      [,6]
[1,] -182.8642 -195.7694 -301.7938 -2530.659 -2973.657 -2177.901
[1] "VaR a nivel contrato"
          [,1]      [,2]      [,3]      [,4]      [,5]      [,6]
[1,] -145.8201 -156.1109 -240.6571 -2018.004 -2371.261 -1736.707
[1] "VaR a nivel contrato CF"
          [,1]      [,2]      [,3]      [,4]      [,5]      [,6]
[1,] -145.8201 -156.1109 -240.6571 -2018.004 -2371.261 -1736.707
[1] "CVaR a nivel contrato"
          [,1]      [,2]      [,3]      [,4]      [,5]      [,6]
[1,] -182.8642 -195.7694 -301.7938 -2530.659 -2973.657 -2177.901
[1] "CVaR a nivel contrato CF"
          [,1]      [,2]

### Riesgo a nivel total y total factor de riesgo

In [ ]:
%%R
#RIESGO TOTAL acciones y divisas
#Divisas

DeltaX_div=DeltaX_port[,1:(sum(n_if[1])/2)]
VarDeltaX_div=cov(DeltaX_div)*(n/(n-1))
#print(VarDeltaX_div)
tgammavar_div=(gamma_acc_divt[1:3,1:3] %*% VarDeltaX_div)
tgammavar2_div=(tgammavar_div)%*%(tgammavar_div) 
mu_div=0
g_divt<-t(as.matrix(g_acc_divt[,1:3]))
mudg_div=ifelse((sum(diag(tgammavar_div))/2+sum(mu_div))>0,0,(sum(diag(tgammavar_div))/2+sum(mu_div)))
desv_div=sqrt(sum(diag(tgammavar2_div))/2+g_divt %*% VarDeltaX_div %*% t(g_divt))

VaRT_div_DGM=(mudg_div)+(desv_div)*qnorm(1-alpha)  #VaR Normal
CVaRT_div_DGM=-(mudg_div)-(desv_div)*dnorm(qnorm(alpha))/(1-alpha)

mudn_div=ifelse((sum(mu_div))>0,0,sum(mu_div))
desvdn_div=sqrt(g_divt%*%VarDeltaX_div%*%t(g_divt))

VaRT_div_DN=(mudn_div)+(desvdn_div)*qnorm(1-alpha)  #VaR Normal
CVaRT_div_DN=-(mudn_div)-(desvdn_div)*dnorm(qnorm(alpha))/(1-alpha)

###VaR Cornish Fisher
tgammavar3_div=tgammavar2_div%*%(tgammavar_div)
asimP_div=sum(diag(tgammavar3_div))+3*g_divt%*%VarDeltaX_div%*%gamma_acc_divt[1:3,1:3]%*%VarDeltaX_div%*%t(g_divt) #gamma tercer momento
asimP_div=asimP_div/(desv_div)^3  #coef de asimetría
VaRstd_div=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_div  #VaR estandarizado
VaRT_div_DGM_CF=(mudg_div)+(desv_div)*VaRstd_div #VaR cf
CVaRT_div_DGM_CF=-(mudg_div)-(desv_div)*dnorm(VaRstd_div)/(pnorm(VaRstd_div))
print("VaR y CVaR DN")
print(VaRT_div_DN)
print(CVaRT_div_DN)
print("VaR y CVaR DGN")
print(VaRT_div_DGM)
print(CVaRT_div_DGM)
print("VaR y CVaR CF")
print(VaRT_div_DGM_CF)
print(CVaRT_div_DGM_CF)

#Acciones
DeltaX_acc=DeltaX_port[,(sum(n_if[1])/2+1):(sum(n_if[1]))]
VarDeltaX_acc=cov(DeltaX_acc)*(n/(n-1))
#print(VarDeltaX_acc)
tgammavar_acc=(gamma_acc_divt[4:6,4:6] %*% VarDeltaX_acc)
tgammavar2_acc=(tgammavar_acc)%*%(tgammavar_acc) 
mu_acc=0
g_acct<-t(as.matrix(g_acc_divt[,4:6]))
mudg_acc=ifelse((sum(diag(tgammavar_acc))/2+sum(mu_acc))>0,0,(sum(diag(tgammavar_acc))/2+sum(mu_acc)))
desv_acc=sqrt(sum(diag(tgammavar2_acc))/2+g_acct%*%VarDeltaX_acc%*%t(g_acct))

VaRT_acc_DGM=(mudg_acc)+(desv_acc)*qnorm(1-alpha)  #VaR Normal
CVaRT_acc_DGM=-(mudg_acc)-(desv_acc)*dnorm(qnorm(alpha))/(1-alpha)

mudn_acc=ifelse((sum(mu_acc))>0,0,sum(mu_acc))
desvdn_acc=sqrt(g_acct%*%VarDeltaX_acc%*%t(g_acct))

VaRT_acc_DN=(mudn_acc)+(desvdn_acc)*qnorm(1-alpha)  #VaR Normal
CVaRT_acc_DN=-(mudn_acc)-(desvdn_acc)*dnorm(qnorm(alpha))/(1-alpha)

###VaR Cornish Fisher
tgammavar3_acc=tgammavar2_acc%*%(tgammavar_acc)
asimP_acc=sum(diag(tgammavar3_acc))+3*g_acct%*%VarDeltaX_acc%*%gamma_acc_divt[1:3,1:3]%*%VarDeltaX_acc%*%t(g_acct) #gamma tercer momento
asimP_acc=asimP_acc/(desv_acc)^3  #coef de asimetría
VaRstd_acc=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_acc  #VaR estandarizado
VaRT_acc_DGM_CF=(mudg_acc)+(desv_acc)*VaRstd_acc #VaR cf
CVaRT_acc_DGM_CF=-(mudg_acc)-(desv_acc)*dnorm(VaRstd_acc)/(pnorm(VaRstd_acc))

print(VaRT_acc_DN)
print(CVaRT_acc_DN)
print(VaRT_acc_DGM)
print(CVaRT_acc_DGM)
print(VaRT_acc_DGM_CF)
print(CVaRT_acc_DGM_CF)



[1] "VaR y CVaR DN"
          [,1]
[1,] -297.9672
         [,1]
[1,] -373.663
[1] "VaR y CVaR DGN"
          [,1]
[1,] -297.9672
         [,1]
[1,] -373.663
[1] "VaR y CVaR CF"
          [,1]
[1,] -297.9672
         [,1]
[1,] -373.663
          [,1]
[1,] -2844.765
         [,1]
[1,] -3567.45
          [,1]
[1,] -2844.765
         [,1]
[1,] -3567.45
          [,1]
[1,] -2844.765
         [,1]
[1,] -3567.45


##CETES

### Riesgo de contrato y nivel contrato-factor de riesgo

In [ ]:
%%R
#Cálculo de matriz de pérdidas y ganancias Bonos M
#Cálculo de VaR por contrato
#creación de matrices de VaR, gamma, VaRCF por contrato
DeltaX_bcc=DeltaX_port[,(sum(n_if[1:1],1)):(sum(n_if[1:2]))] 
m_bcc=length(plazos_bcc)
m=length(plazos_bcc)
VaRmargc_bcc=matrix(0,1,m) #Var por contrato Delta GammaNormal
VaRmargcdn_bcc=matrix(0,1,m) #Var por Delta Normal
VaRmargcstd_bcc=matrix(0,1,m) #Var por contrato estandarizado 
VaRmargcCF_bcc=matrix(0,1,m) #Var por contrato Cornish-Fisher
CVaRmargcdn_bcc=matrix(0,1,m)
CVaRmargc_bcc=matrix(0,1,m) #CVar por contrato estandarizado
CVaRmargcCF_bcc=matrix(0,1,m) #CVar por contrato estandarizado

##factores riesgo
#Spot
VaRmargc_fr_bcc=matrix(0,1,m) #Var por contrato Delta Normal
VaRmargcstd_fr_bcc=matrix(0,1,m) #Var por contrato estandarizado 
VaRmargcCF_fr_bcc=matrix(0,1,m) #Var por contrato Cornish-Fisher
CVaRmargc_fr_bcc=matrix(0,1,m) #CVar por contrato estandarizado
CVaRmargcCF_fr_bcc=matrix(0,1,m) #CVar por contrato estandarizado
WFR=diagv(c(1))

gamma_bcc2=bcc_di2(x0_bcc,plazos_bcc)*contratos_bcc*nominal_bcc
for (i in 1:(m)){
  delta_bcc=t(g_bcc[i]) 
  #print(delta_bcc)
  tdelta_bcc=t(g_bcc[i])*x0_bcc[i]   #delta tilde
  gamma_bcc=gamma_bcc2[i]
  #gamma_bcct2=diagv(gamma_bcc*(x0_bcc[i])^2)  #paso clave
  tgamma_bcc=gamma_bcc*(x0_bcc[i])^2 #gamma tilde
  #print(tgamma_bcc)
  Varmargcaux_bcc=var(DeltaX_bcc[,i])*((n)/(n-1)) #matriz varcov factores de riesgo de cada contrato
  #print(Varmargcaux_bcc)
  tgammavar_bcc=(tgamma_bcc%*%Varmargcaux_bcc)
  tgammavar2_bcc=(tgamma_bcc%*%Varmargcaux_bcc)%*%(tgamma_bcc%*%Varmargcaux_bcc)  
  mudg_bcc=ifelse((sum(diag(tgammavar_bcc))/2+mu_bcc[i])>0,0,(sum(diag(tgammavar_bcc))/2+mu_bcc[i]))
  desv_bcc=sqrt(sum(diag(tgammavar2_bcc))/2+tdelta_bcc%*%Varmargcaux_bcc%*%t(tdelta_bcc))

  VaRmargc_bcc[i]=mudg_bcc+desv_bcc*qnorm(1-alpha) #VaR Normal por contrato
  CVaRmargc_bcc[i]=-mudg_bcc-desv_bcc*(dnorm(qnorm(1-alpha))/(1-alpha))

  #Delta normal
  mudgdn_bcc=ifelse((mu_bcc[i])>0,0,mu_bcc[i]) 
  desvdn_bcc=sqrt(tdelta_bcc%*%Varmargcaux_bcc%*%t(tdelta_bcc))

  VaRmargcdn_bcc[i]=mudgdn_bcc+desvdn_bcc*qnorm(1-alpha) #VaR Normal por contrato
  CVaRmargcdn_bcc[i]=-mudgdn_bcc-desvdn_bcc*(dnorm(qnorm(1-alpha))/(1-alpha))
  
  
  tgammavar3_bcc=tgammavar2_bcc%*%(tgamma_bcc%*%Varmargcaux_bcc)
  asimPmargc_bcc=sum(diag(tgammavar3_bcc))+3*tdelta_bcc%*%Varmargcaux_bcc%*%tgamma_bcc%*%Varmargcaux_bcc%*%t(tdelta_bcc) #tercer momento
  asimPmargc_bcc=asimPmargc_bcc/(desv_bcc)^3  #coef de asimetría
  VaRmargcstd_bcc[i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_bcc  #VaR estandarizado
  VaRmargcCF_bcc[i]=mudg_bcc+desv_bcc*VaRmargcstd_bcc[i] 	    #VaR Cornish-Fisher
  CVaRmargcCF_bcc[i]=-mudg_bcc-desv_bcc*(dnorm(VaRmargcstd_bcc[i])/(pnorm(VaRmargcstd_bcc[i])))
}

print("Valor a nivel contrato")
print(V0_bcc)
print("VaR a nivel contrato DN")
print(VaRmargcdn_bcc)
print("CVaR a nivel contrato DN")
print(CVaRmargcdn_bcc)
print("VaR a nivel contrato")
print(VaRmargc_bcc)
print("VaR a nivel contrato CF")
print(VaRmargcCF_bcc)
print("CVaR a nivel contrato")
print(CVaRmargc_bcc)
print("CVaR a nivel contrato CF")
print(CVaRmargcCF_bcc)



R[write to console]: Error in DeltaX_bcc[, i] : incorrect number of dimensions
Calls: <Anonymous> -> <Anonymous> -> withVisible -> var -> is.data.frame

R[write to console]: In addition: 
R[write to console]: Warning messages:

R[write to console]: 1: AMXL.MX contains missing values. Some functions will not work if objects contain missing values in the middle of the series. Consider using na.omit(), na.approx(), na.fill(), etc to remove or replace them. 

R[write to console]: 2: GCARSOA1.MX contains missing values. Some functions will not work if objects contain missing values in the middle of the series. Consider using na.omit(), na.approx(), na.fill(), etc to remove or replace them. 

R[write to console]: 3: WALMEX.MX contains missing values. Some functions will not work if objects contain missing values in the middle of the series. Consider using na.omit(), na.approx(), na.fill(), etc to remove or replace them. 

R[write to console]: 4: EURUSD=X contains missing values. Some functio


Error in DeltaX_bcc[, i] : incorrect number of dimensions
Calls: <Anonymous> -> <Anonymous> -> withVisible -> var -> is.data.frame


##Bonos M

 ### Riesgo de contrato y nivel contrato-factor de riesgo

In [ ]:
%%R
#Cálculo de matriz de pérdidas y ganancias Bonos M

#Cálculo de VaR por contrato
#creación de matrices de VaR, gamma, VaRCF por contrato
DeltaX_bm=DeltaX_port[,(sum(n_if[1:2],1)):(sum(n_if[1:3]))] 
m_bm=length(plazos_bm)
m=length(plazos_bm)
VaRmargc_bm=matrix(0,1,m) #Var por contrato Delta GammaNormal
VaRmargcdn_bm=matrix(0,1,m) #Var por Delta Normal
VaRmargcstd_bm=matrix(0,1,m) #Var por contrato estandarizado 
VaRmargcCF_bm=matrix(0,1,m) #Var por contrato Cornish-Fisher
CVaRmargcdn_bm=matrix(0,1,m)
CVaRmargc_bm=matrix(0,1,m) #CVar por contrato estandarizado
CVaRmargcCF_bm=matrix(0,1,m) #CVar por contrato estandarizado

##factores riesgo
#Spot
VaRmargc_fr_bm=matrix(0,1,m) #Var por contrato Delta Normal
VaRmargcstd_fr_bm=matrix(0,1,m) #Var por contrato estandarizado 
VaRmargcCF_fr_bm=matrix(0,1,m) #Var por contrato Cornish-Fisher
CVaRmargc_fr_bm=matrix(0,1,m) #CVar por contrato estandarizado
CVaRmargcCF_fr_bm=matrix(0,1,m) #CVar por contrato estandarizado
WFR=diagv(c(1))

gamma_bm2=eval(Deriv(bonoMyield(x0_bm,plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm), "x0_bm",nderiv=2))
print(gamma_bm2)
for (i in 1:(m))
{
  delta_bm=t(g_bm[i]) 
  print(delta_bm)
  tdelta_bm=t(g_bm[i])*x0_bm[i]   #delta tilde
  gamma_bm=gamma_bm2[i]
  #gamma_bmt2=diagv(gamma_bm*(x0_bm[i])^2)  #paso clave
  tgamma_bm=gamma_bm*(x0_bm[i])^2 #gamma tilde
  print(tgamma_bm)
  Varmargcaux_bm=var(DeltaX_bm[,i])*((n)/(n-1)) #matriz varcov factores de riesgo de cada contrato
  print(Varmargcaux_bm)
  tgammavar_bm=(tgamma_bm%*%Varmargcaux_bm)
  tgammavar2_bm=(tgamma_bm%*%Varmargcaux_bm)%*%(tgamma_bm%*%Varmargcaux_bm)  
  mudg_bm=ifelse((sum(diag(tgammavar_bm))/2+mu_bm[i])>0,0,(sum(diag(tgammavar_bm))/2+mu_bm[i]))
  desv_bm=sqrt(sum(diag(tgammavar2_bm))/2+tdelta_bm%*%Varmargcaux_bm%*%t(tdelta_bm))

  VaRmargc_bm[i]=mudg_bm+desv_bm*qnorm(1-alpha) #VaR Normal por contrato
  CVaRmargc_bm[i]=-mudg_bm-desv_bm*(dnorm(qnorm(1-alpha))/(1-alpha))

  #Delta normal
  mudgdn_bm=ifelse((mu_bm[i])>0,0,mu_bm[i]) 
  desvdn_bm=sqrt(tdelta_bm%*%Varmargcaux_bm%*%t(tdelta_bm))

  VaRmargcdn_bm[i]=mudgdn_bm+desvdn_bm*qnorm(1-alpha) #VaR Normal por contrato
  CVaRmargcdn_bm[i]=-mudgdn_bm-desvdn_bm*(dnorm(qnorm(1-alpha))/(1-alpha))
  
  tgammavar3_bm=tgammavar2_bm%*%(tgamma_bm%*%Varmargcaux_bm)
  asimPmargc_bm=sum(diag(tgammavar3_bm))+3*tdelta_bm%*%Varmargcaux_bm%*%tgamma_bm%*%Varmargcaux_bm%*%t(tdelta_bm) #tercer momento
  asimPmargc_bm=asimPmargc_bm/(desv_bm)^3  #coef de asimetría
  VaRmargcstd_bm[i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_bm  #VaR estandarizado
  VaRmargcCF_bm[i]=mudg_bm+desv_bm*VaRmargcstd_bm[i] 	    #VaR Cornish-Fisher
  CVaRmargcCF_bm[i]=-mudg_bm-desv_bm*(dnorm(VaRmargcstd_bm[i])/(pnorm(VaRmargcstd_bm[i])))

#Por factor de riesgo y por contrato
#spot, tasa extranjera, tasa nacional
#Dado que solo se tiene un factor de riesgo eliminamos el for
}

print("Valor a nivel contrato")
print(V0_bm)
print("VaR a nivel contrato DN")
print(VaRmargcdn_bm)
print("CVaR a nivel contrato DN")
print(CVaRmargcdn_bm)
print("VaR a nivel contrato")
print(VaRmargc_bm)
print("VaR a nivel contrato CF")
print(VaRmargcCF_bm)
print("CVaR a nivel contrato")
print(CVaRmargc_bm)
print("CVaR a nivel contrato CF")
print(CVaRmargcCF_bm)




         [,1] [,2]
[1,] 4988.958   NA
          [,1]
[1,] -513.2187
[1] 30.54395
[1] 0.0003039207
     [,1]
[1,]   NA
[1] NA
[1] 2.550257e-05
[1] "Valor a nivel contrato"
         [,1] [,2]
[1,] 66.48752  NaN
[1] "VaR a nivel contrato DN"
          [,1] [,2]
[1,] -1.158214   NA
[1] "CVaR a nivel contrato DN"
          [,1] [,2]
[1,] -1.437335   NA
[1] "VaR a nivel contrato"
          [,1] [,2]
[1,] -1.153623   NA
[1] "VaR a nivel contrato CF"
          [,1] [,2]
[1,] -1.145707   NA
[1] "CVaR a nivel contrato"
         [,1] [,2]
[1,] -1.44204   NA
[1] "CVaR a nivel contrato CF"
          [,1] [,2]
[1,] -1.435221   NA


## Riesgo de Bondes D

### Riesgo a nivel contrato y nivel contrato-factor de riesgo

In [ ]:
%%R
Delta_X_bd_ext=X_bd_ext[1:(n-1),]/X_bd_ext[2:(n),]-1
#print(Delta_X_bd_ext)
##POR INSTRUMENTO
#Para obtener el VaR por contrato
VaRContDGM_bd=matrix(0,1,count(N_bd))
CVaRContDGM_bd=matrix(0,1,count(N_bd))
VaRContCF_bd=matrix(0,1,count(N_bd))
CVaRContCF_bd=matrix(0,1,count(N_bd))
#Para obtener el VaR por contrato por factor de riesgo
VaRContDGM_bdf=matrix(0,3,count(N_bd))
CVaRContDGM_bdf=matrix(0,3,count(N_bd))
VaRContCF_bdf=matrix(0,3,count(N_bd))
CVaRContCF_bdf=matrix(0,3,count(N_bd))
VaRContDN_fr_bd=matrix(0,3,count(N_bd)) 
CVaRContDN_fr_bd=matrix(0,3,count(N_bd))
#V0=swap(porT, contratosT, nominalT, XtfwdT[1,], tasafijaT, plazocuponT, VTplazos, Xvp[1,],N)
 
#Valor de las derivadas evaluadas en el punto x0.
V0f_dtc_bd=d_bondeD_tc(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv)
V0f_dvp_bd=d_bondeD_x(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv)
V0f_dst_bd=V0f_dvp_bd
 
for (j in (1:count(N_bd)))
{
  if(j==1)
  { 
    x0cont_bd=c(X_bd_tc[1,1:sum(N_bd[1:j])],Xvp_bd[1,1:sum(N_bd[1:j])],Xst_bd[1,1:sum(N_bd[1:j])])
    #MATRIZ de factores de riesgo primera fila cupón, segunda valor presente, tercera sobretasa
    x0cont_bdf=rbind(c(X_bd_tc[1,1:sum(N_bd[1:j])],Xvp_bd[1,1:sum(N_bd[1:j])]*0,Xst_bd[1,1:sum(N_bd[1:j])]*0),
                     c(X_bd_tc[1,1:sum(N_bd[1:j])]*0,Xvp_bd[1,1:sum(N_bd[1:j])],Xst_bd[1,1:sum(N_bd[1:j])]*0),
                     c(X_bd_tc[1,1:sum(N_bd[1:j])]*0,Xvp_bd[1,1:sum(N_bd[1:j])]*0,Xst_bd[1,1:sum(N_bd[1:j])]))
    mucont_bd=sum(mu_bd[j])
    gcont_bd=as.matrix(c(V0f_dtc_bd[1:sum(N_bd[1:j])],V0f_dvp_bd[1:sum(N_bd[1:j])],V0f_dst_bd[1:sum(N_bd[1:j])]))
    gtcont_bd=t(as.matrix(gcont_bd*x0cont_bd))
    VarCovXcont_bd=cov(cbind(X_bd_tc[,1:sum(N_bd[1:j])],Xvp_bd[,1:sum(N_bd[1:j])],Xst_bd[,1:sum(N_bd[1:j])]))*(n-1)/(n-2)   
    gamma_cont_bd=gamma_bd[1:(3*sum(N_bd[1:j])),1:(3*sum(N_bd[1:j]))]
    tgamma_bd= x0cont_bd%*%t(x0cont_bd)*gamma_cont_bd #gamma tilde
  }
  else
  {
    x0cont_bd=c(X_bd_tc[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])],Xvp_bd[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])],Xst_bd[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])])
    x0cont_bdf=rbind(c(X_bd_tc[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])],Xvp_bd[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])]*0,Xst_bd[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])]*0),
                     c(X_bd_tc[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])]*0,Xvp_bd[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])],Xst_bd[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])]*0),
                     c(X_bd_tc[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])]*0,Xvp_bd[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])]*0,Xst_bd[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])]))
    mucont_bd=sum(mu_bd[j])
    gcont_bd=as.matrix(c(V0f_dtc_bd[sum(N_bd[1:j-1],1):sum(N_bd[1:j])],V0f_dvp_bd[sum(N_bd[1:j-1],1):sum(N_bd[1:j])],V0f_dst_bd[sum(N_bd[1:j-1],1):sum(N_bd[1:j])]))
    gtcont_bd=t(as.matrix(gcont_bd*x0cont_bd))
    VarCovXcont_bd=cov(cbind(X_bd_tc[,sum(N_bd[1:j-1],1):sum(N_bd[1:j])],Xvp_bd[,sum(N_bd[1:j-1],1):sum(N_bd[1:j])],Xst_bd[,sum(N_bd[1:j-1],1):sum(N_bd[1:j])]))*(n-1)/(n-2)
    gamma_cont_bd=gamma_bd[(3*sum(N_bd[1:j-1])+1):(3*sum(N_bd[1:j])),(3*sum(N_bd[1:j-1])+1):(3*sum(N_bd[1:j]))]
    tgamma_bd= x0cont_bd%*%t(x0cont_bd)*gamma_cont_bd #gamma tilde
  }
  
  
  tgammavar_bd=(tgamma_bd%*%VarCovXcont_bd)
  tgammavar2_bd=(tgamma_bd%*%VarCovXcont_bd)%*%(tgamma_bd%*%VarCovXcont_bd)
  VaRContDGM_bd[j]=ifelse((sum(diag(tgammavar_bd))/2+mucont_bd)>0,0,(sum(diag(tgammavar_bd))/2+mucont_bd))+(sqrt(sum(diag(tgammavar2_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))*qnorm(1-alpha) #VaR Normal
  CVaRContDGM_bd[j]=ifelse((sum(diag(tgammavar_bd))/2+mucont_bd)>0,0,(sum(diag(tgammavar_bd))/2+mucont_bd))-(sqrt(sum(diag(tgammavar_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))*dnorm(qnorm(alpha))/(1-alpha)

#Delta normal
mudn_bd=ifelse(sum(mu_bd)>0,0,sum(mu_bd))
desvdn_bd=sqrt(gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd))
VaRContDN_bd=mudn_bd+desvdn_bd*qnorm(1-alpha) #VaR Normal por contrato Delta Normal
CVaRContDN_bd=mudn_bd-desvdn_bd*(dnorm(qnorm(1-alpha))/(1-alpha))  #CVaR Normal por contrato Delta Normal


  ###VaR Cornish Fisher
  tgammavar3_bd=tgammavar2_bd%*%(tgamma_bd%*%VarCovXcont_bd)
  asimPmargc_bd=sum(diag(tgammavar3_bd))+3*gtcont_bd%*%VarCovXcont_bd%*%tgamma_bd%*%VarCovXcont_bd%*%t(gtcont_bd) #gamma tercer momento
  asimPmargc_bd=asimPmargc_bd/(sqrt(sum(diag(tgammavar2_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))^3  #coef de asimetría
  VaRmargcstd_bd=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_bd  #VaR estandarizado
  VaRContCF_bd[j]=ifelse((sum(diag(tgammavar_bd))/2+mucont_bd)>0,0,(sum(diag(tgammavar_bd))/2+mucont_bd))+(sqrt(sum(diag(tgammavar2_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))*VaRmargcstd_bd #VaR Normal
  CVaRContCF_bd[j]=ifelse((sum(diag(tgammavar_bd))/2+mucont_bd)>0,0,(sum(diag(tgammavar_bd))/2+mucont_bd))-(sqrt(sum(diag(tgammavar2_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))*dnorm(VaRmargcstd_bd)/(pnorm(VaRmargcstd_bd))
  for (k in (1:3))
  {
    gtcont_bd=t(as.matrix(gcont_bd*x0cont_bdf[k,]))
    tgamma_bd= x0cont_bdf[k,]%*%t(x0cont_bdf[k,])*gamma_cont_bd #gamma tilde
    tgammavar_bd=(tgamma_bd%*%VarCovXcont_bd)
    tgammavar2_bd=(tgamma_bd%*%VarCovXcont_bd)%*%(tgamma_bd%*%VarCovXcont_bd)
    VaRContDGM_bdf[k,j]=(sum(diag(tgammavar_bd))/2+mucont_bd)-(sqrt(sum(diag(tgammavar2_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))*qnorm(alpha) #VaR Normal
    CVaRContDGM_bdf[k,j]=(sum(diag(tgammavar_bd))/2+mucont_bd)-(sqrt(sum(diag(tgammavar_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))*dnorm(qnorm(alpha))/(1-alpha)
    #Delta Normal
    mudn_bd=0
    desvdn_bd=sqrt(gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd))
    VaRContDN_fr_bd[j]=mudn_bd+desvdn_bd*qnorm(1-alpha) #VaR Normal por contrato Delta Normal
    CVaRContDN_fr_bd[j]=mudn_bd-desvdn_bd*(dnorm(qnorm(1-alpha))/(1-alpha))  #CVaR Normal por contrato Delta Normal

    ###VaR Cornish Fisher
    tgammavar3_bd=tgammavar2_bd%*%(tgamma_bd%*%VarCovXcont_bd)
    asimPmargc_bd=sum(diag(tgammavar3_bd))+3*gtcont_bd%*%VarCovXcont_bd%*%tgamma_bd%*%VarCovXcont_bd%*%t(gtcont_bd) #gamma tercer momento
    asimPmargc_bd=asimPmargc_bd/(sqrt(sum(diag(tgammavar2_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))^3  #coef de asimetría
    VaRmargcstd_bd=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_bd  #VaR estandarizado
    VaRContCF_bdf[k,j]=(sqrt(sum(diag(tgammavar2_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))*VaRmargcstd_bd #VaR Normal
    CVaRContCF_bdf[k,j]=-(sqrt(sum(diag(tgammavar2_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))*dnorm(VaRmargcstd_bd)/(pnorm(VaRmargcstd_bd))    
  }
  
}
print("VaR contrato DN")
print(VaRContDN_bd) #VaR contrato Delta normal 
print("CVaR contrato DN")
print(CVaRContDN_bd) #CVaR contrato Delta normal 
print("VaR contrato DGN")
print(VaRContDGM_bd)  #VaR contrato Delta Gamma Normal 
print("CVaR contrato DGN")
print(CVaRContDGM_bd) #CVaR contrato Delta Gamma Normal 
print("VaR contrato CF")
print(VaRContCF_bd) #VaR contrato Delta Gamma CF 
print("CVaR contrato CF")
print(CVaRContCF_bd) #CVaR contrato Delta Gamma CF
print("VaR factor DN")
print(VaRContDN_fr_bd)
print("CVaR factor DN")
print(CVaRContDN_fr_bd)
print("VaR factor DGN")
print(VaRContDGM_bdf) #VaR contrato-factor de riesgo Delta Gamma Normal 
print("CVaR factor DGN")
print(CVaRContDGM_bdf) #CVaR contrato-factor de riesgo Delta Gamma Normal 
print("VaR factor CF")
print(VaRContCF_bdf) #VaR contrato-factor de riesgo Delta Gamma CF
print("CVaR factor CF")
print(CVaRContCF_bdf) #CVaR contrato-factor de riesgo Delta Gamma CF

R[write to console]: Error in 1:ddv[j] : NA/NaN argument
Calls: <Anonymous> -> <Anonymous> -> withVisible -> d_bondeD_tc




Error in 1:ddv[j] : NA/NaN argument
Calls: <Anonymous> -> <Anonymous> -> withVisible -> d_bondeD_tc


### Riesgo a nivel total y total factor de riesgo

In [ ]:

%%R
VarCovX_bd_ext=cov(Delta_X_bd_ext)*(n-1)/(n-2)
tgammavar_bdt=(gamma_bdt2%*%VarCovX_bd_ext)
tgammavar2_bdt=(gamma_bdt2%*%VarCovX_bd_ext)%*%(gamma_bdt2%*%VarCovX_bd_ext) 
VaRTotalDGM=ifelse((sum(diag(tgammavar_bdt))/2+sum(mu_bd))>0,0,(sum(diag(tgammavar_bdt))/2+sum(mu_bd)))+(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))*qnorm(1-alpha) #VaR Normal
CVaRTotalDGM=ifelse((sum(diag(tgammavar_bdt))/2+sum(mu_bd))>0,0,(sum(diag(tgammavar_bdt))/2+sum(mu_bd)))-(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))*dnorm(qnorm(alpha))/(1-alpha)

#Delta normal
mudn_bd=ifelse(sum(mu_bd)>0,0,sum(mu_bd))
desvdn_bd=sqrt(g_bdt%*%VarCovX_bd_ext%*%t(g_bdt))
VaRTDN_bd=mudn_bd+desvdn_bd*qnorm(1-alpha) #VaR Normal por contrato Delta Normal
CVaRTDN_bd=mudn_bd-desvdn_bd*(dnorm(qnorm(1-alpha))/(1-alpha))  #CVaR Normal por contrato Delta Normal

###VaR Cornish Fisher
tgammavar3_bdt=tgammavar2_bdt%*%(gamma_bdt2%*%VarCovX_bd_ext) 
asimPmargc_bdt=sum(diag(tgammavar3_bdt))+3*g_bdt%*%VarCovX_bd_ext%*%gamma_bdt2%*%VarCovX_bd_ext%*%t(g_bdt) #gamma tercer momento
asimPmargc_bdt=asimPmargc_bdt/(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))^3  #coef de asimetría
VaRmargcstd_bdt=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_bdt  #VaR estandarizado
VaRTotalCF=ifelse((sum(diag(tgammavar_bdt))/2+sum(mu_bd))>0,0,(sum(diag(tgammavar_bdt))/2+sum(mu_bd)))+(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))*VaRmargcstd_bdt #VaR Normal #VaR cf
CVaRTotalCF=ifelse((sum(diag(tgammavar_bdt))/2+sum(mu_bd))>0,0,(sum(diag(tgammavar_bdt))/2+sum(mu_bd)))-(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))*dnorm(VaRmargcstd_bdt)/(pnorm(VaRmargcstd_bdt))

#Por factor de riesgo
VaRTDN_fr_bd=matrix(0,1,3)
CVaRTDN_fr_bd=matrix(0,1,3)
VaRTotalDGMf=matrix(0,1,3)
CVaRTotalDGMf=matrix(0,1,3)
VaRTotalCFf=matrix(0,1,3)
CVaRTotalCFf=matrix(0,1,3)
 #MATRIZ de factores de riesgo primera fila cupón, segunda valor presente, tercera sobretasa
x0_bd_extf=rbind(cbind(matrix(tf_act,1,ncol(contratos_bdmT)),t(as.matrix(Xvp_bd[1,]))*0,t(as.matrix(Xst_bd[1,]))*0),
                 cbind(matrix(tf_act,1,ncol(contratos_bdmT))*0,t(as.matrix(Xvp_bd[1,])),t(as.matrix(Xst_bd[1,]))*0),
                 cbind(matrix(tf_act,1,ncol(contratos_bdmT))*0,t(as.matrix(Xvp_bd[1,]))*0,t(as.matrix(Xst_bd[1,]))))

for (j in (1:3))
{
g_bdt=g_bd*x0_bd_extf[j,]
gamma_bdt2=gamma_bd*((x0_bd_extf[j,])%*%t(x0_bd_extf[j,]))
VarCovX_bd_ext=cov(Delta_X_bd_ext)*(n-1)/(n-2)
tgammavar_bdt=(gamma_bdt2%*%VarCovX_bd_ext)
tgammavar2_bdt=(gamma_bdt2%*%VarCovX_bd_ext)%*%(gamma_bdt2%*%VarCovX_bd_ext) 
VaRTotalDGMf[,j]=(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))*qnorm(1-alpha) #VaR Normal
CVaRTotalDGMf[,j]=-(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))*dnorm(qnorm(alpha))/(1-alpha)
#Delta Normal
mudn_bd=0
desvdn_bd=sqrt(g_bdt%*%VarCovX_bd_ext%*%t(g_bdt))
VaRTDN_fr_bd[j]=mudn_bd+desvdn_bd*qnorm(1-alpha) #VaR Normal por contrato Delta Normal
CVaRTDN_fr_bd[j]=mudn_bd-desvdn_bd*(dnorm(qnorm(1-alpha))/(1-alpha))  #CVaR Normal por contrato Delta Normal
###VaR Cornish Fisher
tgammavar3_bdt=tgammavar2_bdt%*%(gamma_bdt2%*%VarCovX_bd_ext) 
asimPmargc_bdt=sum(diag(tgammavar3_bdt))+3*g_bdt%*%VarCovX_bd_ext%*%gamma_bdt2%*%VarCovX_bd_ext%*%t(g_bdt) #gamma tercer momento
asimPmargc_bdt=asimPmargc_bdt/(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))^3  #coef de asimetría
VaRmargcstd_bdt=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_bdt  #VaR estandarizado
VaRTotalCFf[,j]=(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))*VaRmargcstd_bdt #VaR Normal #VaR cf
CVaRTotalCFf[,j]=-(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))*dnorm(VaRmargcstd_bdt)/(pnorm(VaRmargcstd_bdt))
}
print(sum(V0_bdm))
print("Var total DN")
print(VaRTDN_bd)
print("CVar total DN")
print(CVaRTDN_bd)
print("Var total DGN")
print(VaRTotalDGM) #VaR total Delta Gamma Normal
print("CVar total DGN")
print(CVaRTotalDGM) #CVaR total Delta Gamma Normal
print("Var total CF")
print(VaRTotalCF) #VaR total Delta Gamma CF
print("CVar total CF")
print(CVaRTotalCF) #CVaR total Delta Gamma CF
print("Var total factor DN")
print(VaRTDN_fr_bd)
print("CVar total factor DN")
print(CVaRTDN_fr_bd)
print("Var total factor DGN")
print(VaRTotalDGMf) #VaR total Delta Gamma Normal por factor de riesgo
print("CVar total factor DGN")
print(CVaRTotalDGMf) #CVaR total Delta Gamma Normal por factor de riesgo
print("Var total factor CF")
print(VaRTotalCFf) #VaR total Delta Gamma CF por factor de riesgo
print("CVar total factor CF")
print(CVaRTotalCFf) #CVaR total Delta Gamma CF por factor de riesgo


[1] -95226.61
[1] "Var total DN"
          [,1]
[1,] -342.5639
[1] "CVar total DN"
         [,1]
[1,] -429.589
[1] "Var total DGN"
          [,1]
[1,] -342.5648
[1] "CVar total DGN"
          [,1]
[1,] -429.5901
[1] "Var total CF"
          [,1]
[1,] -342.8609
[1] "CVar total CF"
          [,1]
[1,] -429.8453
[1] "Var total factor DN"
          [,1]      [,2]      [,3]
[1,] -207.8381 -203.4426 -73.54957
[1] "CVar total factor DN"
          [,1]      [,2]      [,3]
[1,] -260.6374 -255.1252 -92.23413
[1] "Var total factor DGN"
          [,1]     [,2]      [,3]
[1,] -207.8381 -203.443 -73.54959
[1] "CVar total factor DGN"
          [,1]      [,2]      [,3]
[1,] -260.6374 -255.1257 -92.23415
[1] "Var total factor CF"
          [,1]      [,2]      [,3]
[1,] -207.8381 -203.7425 -73.58879
[1] "CVar total factor CF"
          [,1]      [,2]      [,3]
[1,] -260.6374 -255.3839 -92.26794


## Riesgo de Forwards Tipo de Cambio

###Riesgo a nivel contrato y nivel contrato-factor de riesgo

In [ ]:
%%R


#Cálculo de matriz de pérdidas y ganancias FUTUROS TDC

#Cálculo de VaR por contrato
#creación de matrices de VaR, gamma, VaRCF por contrato
DeltaX_ftdc=DeltaX_port[,(sum(n_if[1:4],1)):(sum(n_if[1:5]))] 
m_ftdc=ncol(plazos_fwd)
m=ncol(plazos_fwd)
VaRmargc_ftdcDN=matrix(0,1,m) #Var por contrato Delta Normal
VaRmargc_ftdc=matrix(0,1,m) #Var por contrato Delta gamma Normal
VaRmargcstd_ftdc=matrix(0,1,m) #Var por contrato estandarizado 
VaRmargcCF_ftdc=matrix(0,1,m) #Var por contrato Cornish-Fisher
CVaRmargc_ftdcDN=matrix(0,1,m)#CVaR por contrato DN
CVaRmargc_ftdc=matrix(0,1,m) #CVar por contrato DGN
CVaRmargcCF_ftdc=matrix(0,1,m) #CVar por contrato Cornish-Fisher

##factores riesgo
#Spot
VaRmargc_fr_ftdc=matrix(0,3,m) 
VaRmargcDN_fr_ftdc=matrix(0,3,m)  #Var por contrato Delta Normal
VaRmargcstd_fr_ftdc=matrix(0,3,m) #Var por contrato estandarizado 
VaRmargcCF_fr_ftdc=matrix(0,3,m) #Var por contrato Cornish-Fisher
CVaRmargcDN_fr_ftdc=matrix(0,3,m)
CVaRmargc_fr_ftdc=matrix(0,3,m) #CVar por contrato estandarizado
CVaRmargcCF_fr_ftdc=matrix(0,3,m) #CVar por contrato Cornish-Fisher
WFR=diagv(c(1,1,1))


for (i in 1:(m))
{
  delta_ftdc=t(g_ftdc[seq(i,m*3,by=m)]) 
  tdelta_ftdc=t(g_ftdc[seq(i,m*3,by=m)])*xp_ftdc[seq(i,m*3,by=m)]   #delta tilde
  gamma_ftdc=mgamma_futTC(plazos_fwd[i],X1_fwtdc[1,i],X2_fwtdc[1,i],X3_ftdc[1,],kst_fwd[i],contratos_fwd[i],nominal_fwd)  #paso clave
  tgamma_ftdc=xp_ftdc[seq(i,m*3,by=m)]%*%t(xp_ftdc[seq(i,m*3,by=m)])*gamma_ftdc #gamma tilde
  Varmargcaux_ftdc=cov(DeltaX_ftdc[,seq(i,m*3,by=m)])*((n)/(n-1)) #matriz varcov factores de riesgo de cada contrato
  tgammavar_ftdc=(tgamma_ftdc%*%Varmargcaux_ftdc)
  tgammavar2_ftdc=(tgamma_ftdc%*%Varmargcaux_ftdc)%*%(tgamma_ftdc%*%Varmargcaux_ftdc)  
  mudg_ftdc=ifelse((sum(diag(tgammavar_ftdc))/2+mu_ftdc[i])>0,0,(sum(diag(tgammavar_ftdc))/2+mu_ftdc[i]))
  desv_ftdc=sqrt(sum(diag(tgammavar2_ftdc))/2+tdelta_ftdc%*%Varmargcaux_ftdc%*%t(tdelta_ftdc))

  VaRmargc_ftdc[i]=mudg_ftdc+desv_ftdc*qnorm(1-alpha) #VaR Normal por contrato
  CVaRmargc_ftdc[i]=-mudg_ftdc-desv_ftdc*(dnorm(qnorm(1-alpha))/(1-alpha))
  
 #Delta Normal 
  mudgdn_ftdc=ifelse((mu_ftdc[i])>0,0,mu_ftdc[i]) 
  desvdn_ftdc=sqrt(tdelta_ftdc%*%Varmargcaux_ftdc%*%t(tdelta_ftdc))
  VaRmargc_ftdcDN[i]=mudgdn_ftdc+desvdn_ftdc*qnorm(1-alpha)
  CVaRmargc_ftdcDN[i]=-mudgdn_ftdc-desvdn_ftdc*(dnorm(qnorm(1-alpha))/(1-alpha))

  tgammavar3_ftdc=tgammavar2_ftdc%*%(tgamma_ftdc%*%Varmargcaux_ftdc)
  asimPmargc_ftdc=sum(diag(tgammavar3_ftdc))+3*tdelta_ftdc%*%Varmargcaux_ftdc%*%tgamma_ftdc%*%Varmargcaux_ftdc%*%t(tdelta_ftdc) #tercer momento
  asimPmargc_ftdc=asimPmargc_ftdc/(desv_ftdc)^3  #coef de asimetría
  VaRmargcstd_ftdc[i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_ftdc  #VaR estandarizado
  VaRmargcCF_ftdc[i]=mudg_ftdc+desv_ftdc*VaRmargcstd_ftdc[i] 	    #VaR Cornish-Fisher
  CVaRmargcCF_ftdc[i]=-mudg_ftdc-desv_ftdc*(dnorm(VaRmargcstd_ftdc[i])/(pnorm(VaRmargcstd_ftdc[i])))

#Por factor de riesgo y por contrato
#spot, tasa extranjera, tasa nacional
  for (j in 1:3)
  {
    WFRM=matrix(0,3,3)  #PASO CLAVE
    WFRM[j,j]=1  #PASO CLAVE
    delta_ftdc=t(g_ftdc[seq(i,m*3,by=m)])*WFR[j,] #PASO CLAVE *(1,0,0), (0,1,0), (0,0,1)
    #print(WFR[j,])
    tdelta_ftdc=t(g_ftdc[seq(i,m*3,by=m)])*xp_ftdc[seq(i,m*3,by=m)]*WFR[j,]   #delta tilde #PASO CLAVE
    gamma_ftdc=mgamma_futTC(plazos_fwd[i],X1_fwtdc[1,i],X2_fwtdc[1,i],X3_ftdc[1,],kst_fwd[i],contratos_fwd[i],nominal_fwd)*WFRM  #paso clave
   #print(WFRM)
    tgamma_ftdc=xp_ftdc[seq(i,m*3,by=m)]%*%t(xp_ftdc[seq(i,m*3,by=m)])*gamma_ftdc #gamma tilde
    Varmargcaux_ftdc=cov(DeltaX_ftdc[,seq(i,m*3,by=m)])*((n)/(n-1)) #matriz varcov factores de riesgo de cada contrato
    tgammavar_ftdc=(tgamma_ftdc%*%Varmargcaux_ftdc)
    tgammavar2_ftdc=(tgamma_ftdc%*%Varmargcaux_ftdc)%*%(tgamma_ftdc%*%Varmargcaux_ftdc)  
    #mudg_ftdc=sum(diag(tgammavar_ftdc))/2+mu_ftdc[i]
    mudg_ftdc=0
    desv_ftdc=sqrt(sum(diag(tgammavar2_ftdc))/2+tdelta_ftdc%*%Varmargcaux_ftdc%*%t(tdelta_ftdc))
    VaRmargc_fr_ftdc[j,i]=mudg_ftdc+desv_ftdc*qnorm(1-alpha) #VaR Normal por contrato
    CVaRmargc_fr_ftdc[j,i]=-mudg_ftdc-desv_ftdc*(dnorm(qnorm(1-alpha))/(1-alpha))
    
    #Delta Normal
    mudn_ftdc=0
    desvdn_ftdc=sqrt(tdelta_ftdc%*%Varmargcaux_ftdc%*%t(tdelta_ftdc))
   VaRmargcDN_fr_ftdc[j,i]=mudn_ftdc+desvdn_ftdc*qnorm(1-alpha) #VaR Normal por contrato Delta Normal
    CVaRmargcDN_fr_ftdc[j,i]=mudn_ftdc-desvdn_ftdc*(dnorm(qnorm(1-alpha))/(1-alpha))  #CVaR Normal por contrato Delta Normal

    
    tgammavar3_ftdc=tgammavar2_ftdc%*%(tgamma_ftdc%*%Varmargcaux_ftdc)
    asimPmargc_ftdc=sum(diag(tgammavar3_ftdc))+3*tdelta_ftdc%*%Varmargcaux_ftdc%*%tgamma_ftdc%*%Varmargcaux_ftdc%*%t(tdelta_ftdc) #tercer momento
    asimPmargc_ftdc=asimPmargc_ftdc/(desv_ftdc)^3  #coef de asimetría
    VaRmargcstd_fr_ftdc[j,i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_ftdc  #VaR estandarizado
    VaRmargcCF_fr_ftdc[j,i]=mudg_ftdc+desv_ftdc*VaRmargcstd_fr_ftdc[j,i] 	    #VaR Cornish-Fisher
    CVaRmargcCF_fr_ftdc[j,i]=-mudg_ftdc-desv_ftdc*(dnorm(VaRmargcstd_fr_ftdc[j,i])/(pnorm(VaRmargcstd_fr_ftdc[j,i])))
  }



}

print("Valor a nivel contrato")
print(V0_fwtdc)
print("VaR a nivel contratoDN")
print(VaRmargc_ftdcDN)
print("CVaR a nivel contratoDN")
print(CVaRmargc_ftdcDN)
print("VaR a nivel contrato")
print(VaRmargc_ftdc)
print("VaR a nivel contrato CF")
print(VaRmargcCF_ftdc)
print("CVaR a nivel contrato")
print(CVaRmargc_ftdc)
print("CVaR a nivel contrato CF")
print(CVaRmargcCF_ftdc)

print("VaR a nivel contratoDN")
print(VaRmargcDN_fr_ftdc)
print("CVaR a nivel contrato")
print(CVaRmargcDN_fr_ftdc)
print("VaR a nivel contrato CF")
print(VaRmargcCF_fr_ftdc)
print("CVaR a nivel contrato")
print(CVaRmargc_fr_ftdc)
print("CVaR a nivel contrato CF")
print(CVaRmargcCF_fr_ftdc)


[1] "Valor a nivel contrato"
          [,1]
[1,] -53.23399
[1] "VaR a nivel contratoDN"
          [,1]
[1,] -16.03605
[1] "CVaR a nivel contratoDN"
          [,1]
[1,] -20.10985
[1] "VaR a nivel contrato"
          [,1]
[1,] -16.03605
[1] "VaR a nivel contrato CF"
          [,1]
[1,] -16.03605
[1] "CVaR a nivel contrato"
          [,1]
[1,] -20.10985
[1] "CVaR a nivel contrato CF"
          [,1]
[1,] -20.10985
[1] "VaR a nivel contratoDN"
              [,1]
[1,] -16.035165365
[2,]  -0.006865028
[3,]  -0.014306020
[1] "CVaR a nivel contrato"
              [,1]
[1,] -20.108744284
[2,]  -0.008609023
[3,]  -0.017940327
[1] "VaR a nivel contrato CF"
              [,1]
[1,] -16.035165365
[2,]  -0.006865028
[3,]  -0.014306022
[1] "CVaR a nivel contrato"
              [,1]
[1,] -20.108744284
[2,]  -0.008609023
[3,]  -0.017940327
[1] "CVaR a nivel contrato CF"
              [,1]
[1,] -20.108744284
[2,]  -0.008609022
[3,]  -0.017940328




### Riesgo a nivel total y total factor de riesgo

In [ ]:
%%R
#RIESGO TOTAL FORWARDS DE TIPO DE CAMBIO

VarDeltaX_ftdc=cov(DeltaX_ftdc)*(n/(n-1))

tgammavar_ftdc=(gamma_ftdctT2%*%VarDeltaX_ftdc)
tgammavar2_ftdc=(tgammavar_ftdc)%*%(tgammavar_ftdc) 
mudg_ftdc=ifelse((sum(diag(tgammavar_ftdc))/2+sum(mu_ftdc))>0,0,(sum(diag(tgammavar_ftdc))/2+sum(mu_ftdc)))
desv_ftdc=sqrt(sum(diag(tgammavar2_ftdc))/2+g_ftdctT%*%VarDeltaX_ftdc%*%t(g_ftdctT))

VaRT_ftdc_DGM=(mudg_ftdc)+(desv_ftdc)*qnorm(1-alpha)  #VaR Normal
CVaRT_ftdc_DGM=-(mudg_ftdc)-(desv_ftdc)*dnorm(qnorm(alpha))/(1-alpha)

#Delta normal
mudn_ftdc=ifelse(sum(mu_ftdc)>0,0,sum(mu_ftdc))
desvdn_ftdc=sqrt(g_ftdctT%*%VarDeltaX_ftdc%*%t(g_ftdctT))
VaRTDN_ftdc=mudn_ftdc+desvdn_ftdc*qnorm(1-alpha) #VaR Normal por contrato Delta Normal
CVaRTDN_ftdc=mudn_ftdc-desvdn_ftdc*(dnorm(qnorm(1-alpha))/(1-alpha))  #CVaR Normal por contrato Delta Normal


###VaR Cornish Fisher
tgammavar3_ftdc=tgammavar2_ftdc%*%(tgammavar_ftdc)
asimP_ftdc=sum(diag(tgammavar3_ftdc))+3*g_ftdctT%*%VarDeltaX_ftdc%*%gamma_ftdctT2%*%VarDeltaX_ftdc%*%t(g_ftdctT) #gamma tercer momento
asimP_ftdc=asimP_ftdc/(desv_ftdc)^3  #coef de asimetría
VaRstd_ftdc=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_ftdc  #VaR estandarizado
VaRT_ftdc_DGM_CF=(mudg_ftdc)+(desv_ftdc)*VaRstd_ftdc #VaR cf
CVaRT_ftdc_DGM_CF=-(mudg_ftdc)-(desv_ftdc)*dnorm(VaRstd_ftdc)/(pnorm(VaRstd_ftdc))

print(VaRTDN_ftdc)
print(CVaRTDN_ftdc)
print(VaRT_ftdc_DGM)
print(CVaRT_ftdc_DGM)
print(VaRT_ftdc_DGM_CF)
print(CVaRT_ftdc_DGM_CF)

VaRTDN_fr_ftdc=matrix(0,1,3)
CVaRTDN_fr_ftdc=matrix(0,1,3)
VaRT_fr_ftdc_DGM=matrix(0,1,3)
CVaRT_fr_ftdc_DGM=matrix(0,1,3)
VaRT_fr_ftdc_DGM_CF=matrix(0,1,3)
CVaRT_fr_ftdc_DGM_CF=matrix(0,1,3)

WFRT=matrix(0,3,(m*3))
# WFRT[1,1:m]=matrix(1,1,m)
# WFRT[2,(m+1):(2*m)]=matrix(1,1,m)
# WFRT[3,(2*m+1):(3*m)]=matrix(1,1,m)
#print(WFRT)

#RIESGO TOTAL POR FACTOR DE RIESGO DE FORWARDS DE TIPO DE CAMBIO

for (j in (1:3))
{
WFRT[j,((j-1)*m+1):(j*m)]=matrix(1,1,m)    #Matriz de diseño para 
WFRM=matrix(0,(m*3),(m*3))  #PASO CLAVE
WFRM[((j-1)*m+1):(m*j),((j-1)*m+1):(m*j)]=matrix(1,m,m)  #PASO CLAVE    
gamma_ftdctT2_fr=gamma_ftdctT2*WFRM #evaluación de gamma multiplicado por x^{2}
g_ftdctT_fr=g_ftdctT*WFRT[j,]
# print(WFRM)
# print(g_ftdctT_fr)
# print(gamma_ftdctT2_fr)
# print(WFRT)
tgammavar_ftdc=(gamma_ftdctT2_fr%*%VarDeltaX_ftdc)
tgammavar2_ftdc=(tgammavar_ftdc)%*%(tgammavar_ftdc) 
#mudg_ftdc=sum(diag(tgammavar_ftdc))/2+sum(mu_ftdc)
mudg_ftdc=0
desv_ftdc=sqrt(sum(diag(tgammavar2_ftdc))/2+g_ftdctT_fr%*%VarDeltaX_ftdc%*%t(g_ftdctT_fr))
VaRT_fr_ftdc_DGM[j]=(mudg_ftdc)+(desv_ftdc)*qnorm(1-alpha) #VaR Normal
CVaRT_fr_ftdc_DGM[j]=-(mudg_ftdc)-(desv_ftdc)*dnorm(qnorm(alpha))/(1-alpha)

#Delta Normal
mudn_ftdc=0
desvdn_ftdc=sqrt(g_ftdctT_fr%*%VarDeltaX_ftdc%*%t(g_ftdctT_fr))
VaRTDN_fr_ftdc[j]=mudn_ftdc+desvdn_ftdc*qnorm(1-alpha) #VaR Normal por contrato Delta Normal
CVaRTDN_fr_ftdc[j]=mudn_ftdc-desvdn_ftdc*(dnorm(qnorm(1-alpha))/(1-alpha))  #CVaR Normal por contrato Delta Normal


###VaR Cornish Fisher
tgammavar3_ftdc=tgammavar2_ftdc%*%(tgammavar_ftdc)
asimP_ftdc=sum(diag(tgammavar3_ftdc))+3*g_ftdctT_fr%*%VarDeltaX_ftdc%*%gamma_ftdctT2_fr%*%VarDeltaX_ftdc%*%t(g_ftdctT_fr) #gamma tercer momento
 #print(asimP_ftdc)
asimP_ftdc=asimP_ftdc/(desv_ftdc)^3  #coef de asimetría
 
VaRstd_ftdc=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_ftdc  #VaR estandarizado
#print(VaRstd_ftdc)
VaRT_fr_ftdc_DGM_CF[j]=(mudg_ftdc)+(desv_ftdc)*VaRstd_ftdc #VaR cf
CVaRT_fr_ftdc_DGM_CF[j]=-(mudg_ftdc)-(desv_ftdc)*dnorm(VaRstd_ftdc)/(pnorm(VaRstd_ftdc))


}

print(VaRTDN_fr_ftdc)
print(CVaRTDN_fr_ftdc)
print(VaRT_fr_ftdc_DGM)
print(CVaRT_fr_ftdc_DGM)
print(VaRT_fr_ftdc_DGM_CF)
print(CVaRT_fr_ftdc_DGM_CF)

print(sum(V0_fwtdc))


# cbind(VaRTotal_fwtdc,sum(V0_fwtdc), VaRCont_fwtdc, V0_fwtdc)
# cbind(CVaRTotal_fwtdc,sum(V0_fwtdc), CVaRCont_fwtdc, V0_fwtdc)
# cbind(VaRTotal_fwtdc,VaRTotalfr1_fwtdc,VaRTotalfr2_fwtdc,VaRTotalfr3_fwtdc)
# cbind(CVaRTotal_fwtdc,CVaRTotalfr1_fwtdc,CVaRTotalfr2_fwtdc,CVaRTotalfr3_fwtdc)


          [,1]
[1,] -16.03605
          [,1]
[1,] -20.10985
          [,1]
[1,] -16.03605
          [,1]
[1,] -20.10985
          [,1]
[1,] -16.03605
          [,1]
[1,] -20.10985
          [,1]         [,2]        [,3]
[1,] -16.03517 -0.006865028 -0.01430602
          [,1]         [,2]        [,3]
[1,] -20.10874 -0.008609023 -0.01794033
          [,1]         [,2]        [,3]
[1,] -16.03517 -0.006865028 -0.01430602
          [,1]         [,2]        [,3]
[1,] -20.10874 -0.008609023 -0.01794033
          [,1]         [,2]        [,3]
[1,] -16.03517 -0.006865028 -0.01430602
          [,1]         [,2]        [,3]
[1,] -20.10874 -0.008609022 -0.01794033
[1] -53.23399


## Riesgo de Forwards Indices

###Riesgo a nivel contrato y nivel contrato-factor de riesgo

In [ ]:
%%R

DeltaX_find=DeltaX_port[,(sum(n_if[1:5],1)):(sum(n_if[1:6]))] 
m=ncol(plazos_fwd_ind)
m_find=ncol(plazos_fwd_ind)
VaRmargc_find=matrix(0,1,m) 
VaRmargc_findDN=matrix(0,1,m)#Var por contrato Delta Normal
VaRmargcstd_find=matrix(0,1,m) #Var por contrato estandarizado 
VaRmargcCF_find=matrix(0,1,m) #Var por contrato Cornish-Fisher
CVaRmargc_findDN=matrix(0,1,m)
CVaRmargc_find=matrix(0,1,m) #CVar por contrato estandarizado
CVaRmargcCF_find=matrix(0,1,m) #CVar por contrato estandarizado

##factores riesgo
#Spot
VaRmargc_fr_find=matrix(0,3,m)
VaRmargcDN_fr_find=matrix(0,3,m) #Var por contrato Delta Normal
VaRmargcstd_fr_find=matrix(0,3,m) #Var por contrato estandarizado 
VaRmargcCF_fr_find=matrix(0,3,m) #Var por contrato Cornish-Fisher
CVaRmargcDN_fr_find=matrix(0,3,m)
CVaRmargc_fr_find=matrix(0,3,m) #CVar por contrato estandarizado
CVaRmargcCF_fr_find=matrix(0,3,m) #CVar por contrato estandarizado
WFR=diagv(c(1,1,1))


for (i in 1:(m))
{
  delta_find=t(g_find[seq(i,m*3,by=m)]) 
  tdelta_find=t(g_find[seq(i,m*3,by=m)])*xp_find[seq(i,m*3,by=m)]   #delta tilde
  gamma_find=mgamma_futI(plazos_fwd_ind[i],X1_fwind[1,i],X2_fwind[1,i],X3_find[1,],kst_fwd_ind[i],contratos_fwd_ind[i],nominal_fwd_ind)  #paso clave
  tgamma_find=xp_find[seq(i,m*3,by=m)]%*%t(xp_find[seq(i,m*3,by=m)])*gamma_find #gamma tilde
  Varmargcaux_find=cov(DeltaX_find[,seq(i,m*3,by=m)])*((n)/(n-1)) #matriz varcov factores de riesgo de cada contrato
  tgammavar_find=(tgamma_find%*%Varmargcaux_find)
  tgammavar2_find=(tgamma_find%*%Varmargcaux_find)%*%(tgamma_find%*%Varmargcaux_find)  
  mudg_find=ifelse((sum(diag(tgammavar_find))/2+mu_find[i])>0,0,(sum(diag(tgammavar_find))/2+mu_find[i]))
  #mudg_find=0
  desv_find=sqrt(sum(diag(tgammavar2_find))/2+tdelta_find%*%Varmargcaux_find%*%t(tdelta_find))

  VaRmargc_find[i]=mudg_find+desv_find*qnorm(1-alpha) #VaR Normal por contrato
  CVaRmargc_find[i]=-mudg_find-desv_find*(dnorm(qnorm(1-alpha))/(1-alpha))
  
#Delta Normal 
mudgdn_find=ifelse((mu_find[i])>0,0,mu_find[i]) 
desvdn_find=sqrt(tdelta_find%*%Varmargcaux_find%*%t(tdelta_find))
VaRmargc_findDN[i]=mudgdn_find+desvdn_find*qnorm(1-alpha)
CVaRmargc_findDN[i]=-mudgdn_find-desvdn_find*(dnorm(qnorm(1-alpha))/(1-alpha))

  tgammavar3_find=tgammavar2_find%*%(tgamma_find%*%Varmargcaux_find)
  asimPmargc_find=sum(diag(tgammavar3_find))+3*tdelta_find%*%Varmargcaux_find%*%tgamma_find%*%Varmargcaux_find%*%t(tdelta_find) #tercer momento
  asimPmargc_find=asimPmargc_find/(desv_find)^3  #coef de asimetría
  VaRmargcstd_find[i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_find  #VaR estandarizado
  VaRmargcCF_find[i]=mudg_find+desv_find*VaRmargcstd_find[i] 	    #VaR Cornish-Fisher
  CVaRmargcCF_find[i]=-mudg_find-desv_find*(dnorm(VaRmargcstd_find[i])/(pnorm(VaRmargcstd_find[i])))

#Por factor de riesgo y por contrato

  for (j in 1:3)
  {
    WFRM=matrix(0,3,3)  #PASO CLAVE
    WFRM[j,j]=1  #PASO CLAVE
    delta_find=t(g_find[seq(i,m*3,by=m)])*WFR[j,] #PASO CLAVE
    tdelta_find=t(g_find[seq(i,m*3,by=m)])*xp_find[seq(i,m*3,by=m)]*WFR[j,]   #delta tilde #PASO CLAVE
  	gamma_find=mgamma_futI(plazos_fwd_ind[i],X1_fwind[1,i],X2_fwind[1,i],X3_find[1,],kst_fwd_ind[i],contratos_fwd_ind[i],nominal_fwd_ind)  #paso clave
    tgamma_find=xp_find[seq(i,m*3,by=m)]%*%t(xp_find[seq(i,m*3,by=m)])*gamma_find #gamma tilde
    Varmargcaux_find=cov(DeltaX_find[,seq(i,m*3,by=m)])*((n)/(n-1)) #matriz varcov factores de riesgo de cada contrato
    tgammavar_find=(tgamma_find%*%Varmargcaux_find)
    tgammavar2_find=(tgamma_find%*%Varmargcaux_find)%*%(tgamma_find%*%Varmargcaux_find)  
    #mudg_find=sum(diag(tgammavar_find))/2+mu_find[i]
   mudg_find=0
    desv_find=sqrt(sum(diag(tgammavar2_find))/2+tdelta_find%*%Varmargcaux_find%*%t(tdelta_find))
    VaRmargc_fr_find[j,i]=mudg_find+desv_find*qnorm(1-alpha) #VaR Normal por contrato
    CVaRmargc_fr_find[j,i]=-mudg_find-desv_find*(dnorm(qnorm(1-alpha))/(1-alpha))
    
#Delta Normal
    mudn_find=0
desvdn_find=sqrt(tdelta_find%*%Varmargcaux_find%*%t(tdelta_find))
VaRmargcDN_fr_find[j,i]=mudn_find+desvdn_find*qnorm(1-alpha) #VaR Normal por contrato Delta Normal
CVaRmargcDN_fr_find[j,i]=mudn_find-desvdn_find*(dnorm(qnorm(1-alpha))/(1-alpha))  #CVaR Normal por contrato Delta Normal

    tgammavar3_find=tgammavar2_find%*%(tgamma_find%*%Varmargcaux_find)
    asimPmargc_find=sum(diag(tgammavar3_find))+3*tdelta_find%*%Varmargcaux_find%*%tgamma_find%*%Varmargcaux_find%*%t(tdelta_find) #tercer momento
    asimPmargc_find=asimPmargc_find/(desv_find)^3  #coef de asimetría
    VaRmargcstd_fr_find[j,i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_find  #VaR estandarizado
    VaRmargcCF_fr_find[j,i]=mudg_find+desv_find*VaRmargcstd_fr_find[j,i] 	    #VaR Cornish-Fisher
    CVaRmargcCF_fr_find[j,i]=-mudg_find-desv_find*(dnorm(VaRmargcstd_fr_find[j,i])/(pnorm(VaRmargcstd_fr_find[j,i])))
  }
}

print("Valor a nivel contrato")
print(V0_fwind)
print("VaR a nivel contratoDN")
print(VaRmargc_findDN)
print("CVaR a nivel contratoDN")
print(CVaRmargc_findDN)
print("VaR a nivel contrato")
print(VaRmargc_find)
print("VaR a nivel contrato CF")
print(VaRmargcCF_find)
print("CVaR a nivel contrato")
print(CVaRmargc_find)
print("CVaR a nivel contrato CF")
print(CVaRmargcCF_find)

print("VaR a nivel contratoDN")
print(VaRmargcDN_fr_find)
print("VaR a nivel contrato")
print(CVaRmargcDN_fr_find)
print("VaR a nivel contrato")
print(VaRmargc_fr_find)
print("VaR a nivel contrato CF")
print(VaRmargcCF_fr_find)
print("CVaR a nivel contrato")
print(CVaRmargc_fr_find)
print("CVaR a nivel contrato CF")
print(CVaRmargcCF_fr_find)

[1] "Valor a nivel contrato"
          [,1]
[1,] -367656.2
[1] "VaR a nivel contratoDN"
          [,1]
[1,] -35488.37
[1] "CVaR a nivel contratoDN"
          [,1]
[1,] -44503.85
[1] "VaR a nivel contrato"
          [,1]
[1,] -35488.37
[1] "VaR a nivel contrato CF"
          [,1]
[1,] -35487.45
[1] "CVaR a nivel contrato"
          [,1]
[1,] -44503.85
[1] "CVaR a nivel contrato CF"
          [,1]
[1,] -44503.06
[1] "VaR a nivel contratoDN"
           [,1]
[1,] -35507.188
[2,]      0.000
[3,]   -297.731
[1] "VaR a nivel contrato"
            [,1]
[1,] -44527.4456
[2,]      0.0000
[3,]   -373.3667
[1] "VaR a nivel contrato"
             [,1]
[1,] -35507.19070
[2,]    -15.06832
[3,]   -298.11210
[1] "VaR a nivel contrato CF"
             [,1]
[1,] -35506.14030
[2,]    -14.02108
[3,]   -297.06171
[1] "CVaR a nivel contrato"
             [,1]
[1,] -44527.44963
[2,]    -18.89628
[3,]   -373.84461
[1] "CVaR a nivel contrato CF"
             [,1]
[1,] -44526.54426
[2,]    -17.99827
[3,]   -372.

### Riesgo a nivel total y total factor de riesgo

In [ ]:
%%R
#RIESGO TOTAL FORWARDS DE TIPO DE CAMBIO
VarDeltaX_find=cov(DeltaX_find)*(n/(n-1))

tgammavar_find=(gamma_findtT2%*%VarDeltaX_find)
tgammavar2_find=(tgammavar_find)%*%(tgammavar_find) 
#media
mudg_find=ifelse((sum(diag(tgammavar_find))/2+sum(mu_find))>0,0,(sum(diag(tgammavar_find))/2+mu_find[i]))
#desviación estándar
desv_find=sqrt(sum(diag(tgammavar2_find))/2+g_findtT%*%VarDeltaX_find%*%t(g_findtT))

VaRT_find_DGM=(mudg_find)+(desv_find)*qnorm(1-alpha)  #VaR Normal
CVaRT_find_DGM=-(mudg_find)-(desv_find)*dnorm(qnorm(alpha))/(1-alpha)

#Delta normal
mudn_find=ifelse(sum(mu_find)>0,0,sum(mu_find))
desvdn_find=sqrt(g_findtT%*%VarDeltaX_find%*%t(g_findtT))
VaRTDN_find=mudn_find+desvdn_find*qnorm(1-alpha) #VaR Normal por contrato Delta Normal
CVaRTDN_find=mudn_find-desvdn_find*(dnorm(qnorm(1-alpha))/(1-alpha))  #CVaR Normal por contrato Delta Normal


###VaR Cornish Fisher
tgammavar3_find=tgammavar2_find%*%(tgammavar_find)
asimP_find=sum(diag(tgammavar3_find))+3*g_findtT%*%VarDeltaX_find%*%gamma_findtT2%*%VarDeltaX_find%*%t(g_findtT) #gamma tercer momento
asimP_find=asimP_find/(desv_find)^3  #coef de asimetría
VaRstd_find=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_find  #VaR estandarizado
VaRT_find_DGM_CF=(mudg_find)+(desv_find)*VaRstd_find #VaR cf
CVaRT_find_DGM_CF=-(mudg_find)-(desv_find)*dnorm(VaRstd_find)/(pnorm(VaRstd_find))

print(VaRTDN_find)
print(CVaRTDN_find)
print(VaRT_find_DGM)
print(CVaRT_find_DGM)
print(VaRT_find_DGM_CF)
print(CVaRT_find_DGM_CF)

VaRTDN_fr_find=matrix(0,1,3)
CVaRTDN_fr_find=matrix(0,1,3)
VaRT_fr_find_DGM=matrix(0,1,3)
CVaRT_fr_find_DGM=matrix(0,1,3)
VaRT_fr_find_DGM_CF=matrix(0,1,3)
CVaRT_fr_find_DGM_CF=matrix(0,1,3)

WFRT=matrix(0,3,(m*3))

#RIESGO TOTAL POR FACTOR DE RIESGO DE FORWARDS DE TIPO DE CAMBIO

for (j in (1:3))
{
WFRT[j,((j-1)*m+1):(j*m)]=matrix(1,1,m)    #Matriz de diseño para 
WFRM=matrix(0,(m*3),(m*3))  #PASO CLAVE
WFRM[((j-1)*m+1):(m*j),((j-1)*m+1):(m*j)]=matrix(1,m,m)  #PASO CLAVE    
gamma_findtT2_fr=gamma_findtT2*WFRM #evaluación de gamma multiplicado por x^{2}
g_findtT_fr=g_findtT*WFRT[j,]
tgammavar_find=(gamma_findtT2_fr%*%VarDeltaX_find)
tgammavar2_find=(tgammavar_find)%*%(tgammavar_find) 
#mudg_find=sum(diag(tgammavar_find))/2+sum(mu_find)
mudg_find=0
desv_find=sqrt(sum(diag(tgammavar2_find))/2+g_findtT_fr%*%VarDeltaX_find%*%t(g_findtT_fr))
VaRT_fr_find_DGM[j]=(mudg_find)+(desv_find)*qnorm(1-alpha) #VaR Normal
CVaRT_fr_find_DGM[j]=-(mudg_find)-(desv_find)*dnorm(qnorm(alpha))/(1-alpha)

#Delta Normal
mudn_find=0
desvdn_find=sqrt(g_findtT_fr%*%VarDeltaX_find%*%t(g_findtT_fr))
VaRTDN_fr_find[j]=mudn_find+desvdn_find*qnorm(1-alpha) #VaR Normal por contrato Delta Normal
CVaRTDN_fr_find[j]=mudn_find-desvdn_find*(dnorm(qnorm(1-alpha))/(1-alpha))  #CVaR Normal por contrato Delta Normal


###VaR Cornish Fisher
tgammavar3_find=tgammavar2_find%*%(tgammavar_find)
asimP_find=sum(diag(tgammavar3_find))+3*g_findtT_fr%*%VarDeltaX_find%*%gamma_findtT2_fr%*%VarDeltaX_find%*%t(g_findtT_fr) #gamma tercer momento
asimP_find=asimP_find/(desv_find)^3  #coef de asimetría
 
VaRstd_find=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_find  #VaR estandarizado
VaRT_fr_find_DGM_CF[j]=(mudg_find)+(desv_find)*VaRstd_find #VaR cf
CVaRT_fr_find_DGM_CF[j]=-(mudg_find)-(desv_find)*dnorm(VaRstd_find)/(pnorm(VaRstd_find))
    
}

print(VaRTDN_fr_find)
print(CVaRTDN_fr_find)
print(VaRT_fr_find_DGM)
print(CVaRT_fr_find_DGM)
print(VaRT_fr_find_DGM_CF)
print(CVaRT_fr_find_DGM_CF)
sum(V0_fwind)


          [,1]
[1,] -35488.37
          [,1]
[1,] -44503.85
          [,1]
[1,] -35488.37
          [,1]
[1,] -44503.85
          [,1]
[1,] -35487.45
          [,1]
[1,] -44503.06
          [,1] [,2]     [,3]
[1,] -35507.19    0 -297.731
          [,1] [,2]      [,3]
[1,] -44527.45    0 -373.3667
          [,1] [,2]     [,3]
[1,] -35507.19    0 -297.731
          [,1] [,2]      [,3]
[1,] -44527.45    0 -373.3667
          [,1] [,2]     [,3]
[1,] -35507.19  NaN -297.731
          [,1] [,2]      [,3]
[1,] -44527.45  NaN -373.3667
[1] -367656.2


##Riesgo de Swaps

###Riesgo a nivel de contrato y nivel contrato-factor de riesgo 

In [ ]:

%%R

#Cálculo de VaR por contrato
#creación de matrices de VaR, gamma, VaRCF por contrato
DeltaX_sw=DeltaX_port[,(sum(n_if[1:6],1)):(sum(n_if[1:7]))] 
m=ncol(plazos_sw)
#VaR y CVaR Delta Normal
VaRcont=matrix(0,1,ncol(N))
CVaRcont=matrix(0,1,ncol(N))
#VaR y CVaR Delta  Gamma Normal
VaRContDGM =matrix(0,1,ncol(N))
CVaRContDGM =matrix(0,1,ncol(N))
#VaR y CVaR Cornish Fisher
VaRContCF =matrix(0,1,ncol(N))
CVaRContCF=matrix(0,1,ncol(N))


VaRcontfr=matrix(0,2,ncol(N))
CVaRcontfr=matrix(0,2,ncol(N))
#VaR y CVaR Delta  Gamma Normal
VaRContfrDGM =matrix(0,2,ncol(N))
CVaRContfrDGM =matrix(0,2,ncol(N))
#VaR y CVaR Cornish Fisher
VaRContfrCF =matrix(0,2,ncol(N))
CVaRContfrCF=matrix(0,2,ncol(N))
WFRcont <- rbind(c(1,0), c(0,1))



g_dtc=dswap_dtc(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,])
g_dvp=dswap_dvp(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,])
DeltaXvp = as.matrix(log(Xvp[1:(n-1),]/Xvp[2:n]))
DeltaXvp[is.na(DeltaXvp)]<-0
DeltaXvp[is.nan(DeltaXvp)]<-0
DeltaXvp[is.infinite(DeltaXvp)]<-0

DeltaXtfwdt = as.matrix(log(XtfwdT[1:(n-1),]/XtfwdT[2:n]))
DeltaXtfwdt[is.na(DeltaXtfwdt)]<-0
DeltaXtfwdt[is.nan(DeltaXtfwdt)]<-0
DeltaXtfwdt[is.infinite(DeltaXtfwdt)]<-0

for (j in (1:ncol(N)))
{
  if(j==1)
  {
    mucont=sum(mu_sw[1:sum(N[1:j])])
    gcont=as.matrix(c(g_dtc[1:sum(N[1:j])],g_dvp[1:sum(N[1:j])]))
    gtcont=t(as.matrix(gcont*c(XtfwdT[1,1:sum(N[1:j])],Xvp[1,1:sum(N[1:j])])))
    VarCovXcont=cov(cbind(DeltaXtfwdt[,1:sum(N[1:j])],DeltaXvp[,1:sum(N[1:j])]))*(n/(n-1))	
    gamma_cont=gamma_sw[1:(2*sum(N[1:j])),1:(2*sum(N[1:j]))]
    tgamma= c(XtfwdT[1,1:sum(N[1:j])],Xvp[1,1:sum(N[1:j])])%*%t(c(XtfwdT[1,1:sum(N[1:j])],Xvp[1,1:sum(N[1:j])]))*gamma_cont #gamma tilde
  }
  else
  {
    mucont=sum(mu_sw[sum(N[1:j-1]+1):sum(N[1:j])])
    gcont=as.matrix(c(g_dtc[sum(N[1:j-1]+1):sum(N[1:j])],g_dvp[sum(N[1:j-1]+1):sum(N[1:j])]))
    gtcont=t(as.matrix(gcont*c(XtfwdT[1,sum(N[1:j-1]+1):sum(N[1:j])],Xvp[1,sum(N[1:j-1]+1):sum(N[1:j])])))
    VarCovXcont=cov(cbind(DeltaXtfwdt[,sum(N[1:j-1]+1):sum(N[1:j])],DeltaXvp[,sum(N[1:j-1]+1):sum(N[1:j])]))*(n/(n-1))
    gamma_cont=gamma_sw[(2*sum(N[1:j-1])+1):(2*sum(N[1:j])),(2*sum(N[1:j-1])+1):(2*sum(N[1:j]))]
    tgamma= c(XtfwdT[1,sum(N[1:j-1]+1):sum(N[1:j])],Xvp[1,sum(N[1:j-1]+1):sum(N[1:j])])%*%t(c(XtfwdT[1,sum(N[1:j-1]+1):sum(N[1:j])],Xvp[1,sum(N[1:j-1]+1):sum(N[1:j])]))*gamma_cont #gamma tilde
  }
  
  
  tgammavar=(tgamma%*%VarCovXcont)
  tgammavar2=(tgamma%*%VarCovXcont)%*%(tgamma%*%VarCovXcont)
  VaRContDGM[j]=(sum(diag(tgammavar))/2+mucont)-(sqrt(sum(diag(tgammavar2))/2+gtcont%*%VarCovXcont%*%t(gtcont)))*qnorm(alpha) #VaR Normal
  CVaRContDGM[j]=(sum(diag(tgammavar))/2+mucont)-(sqrt(sum(diag(tgammavar))/2+gtcont%*%VarCovXcont%*%t(gtcont)))*dnorm(qnorm(alpha))/(1-alpha)
  #VaR Cornish Fisher
  tgammavar3=tgammavar2%*%(tgamma%*%VarCovXcont)
  asimPmargc=sum(diag(tgammavar3))+3*gtcont%*%VarCovXcont%*%tgamma%*%VarCovXcont%*%t(gtcont) #gamma tercer momento
  asimPmargc=asimPmargc/(sqrt(sum(diag(tgammavar2))/2+gtcont%*%VarCovXcont%*%t(gtcont)))^3  #coef de asimetría
  VaRmargcstd=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc  #VaR estandarizado
  VaRContCF[j]=(sum(diag(tgammavar))/2+mucont)+(sqrt(sum(diag(tgammavar2))/2+gtcont%*%VarCovXcont%*%t(gtcont)))*VaRmargcstd #VaR Normal
  CVaRContCF[j]=(sum(diag(tgammavar))/2+sum(mucont))-(sqrt(sum(diag(tgammavar2))/2+gtcont%*%VarCovXcont%*%t(gtcont)))*dnorm(VaRmargcstd)/(pnorm(VaRmargcstd))
  
  #VaR Delta Normal
  VaRcont[j]=sum(mucont)-sqrt(gtcont%*%VarCovXcont%*%t(gtcont))*qnorm(alpha)
  CVaRcont[j]=sum(mucont)-sqrt(gtcont%*%VarCovXcont%*%t(gtcont))*dnorm(qnorm(alpha))/(1-alpha)
}


#contato factor de riesgo
for (j in (1:ncol(N)))
{
  for(k in (1:2)){
    if(j==1)
    {
      mucont=sum(mu_sw[1:sum(N[1:j])])
      gcont=as.matrix(c(g_dtc[1:sum(N[1:j])],g_dvp[1:sum(N[1:j])]))
      gtcont=t(as.matrix(gcont*c(XtfwdT[1,1:sum(N[1:j])]*WFRcont[k,1],Xvp[1,1:sum(N[1:j])]*WFRcont[k,2])))
      VarCovXcont=cov(cbind(DeltaXtfwdt[,1:sum(N[1:j])],DeltaXvp[,1:sum(N[1:j])]))*(n/(n-1))	
      gamma_cont=gamma_sw[1:(2*sum(N[1:j])),1:(2*sum(N[1:j]))]
      tgamma= c(XtfwdT[1,1:sum(N[1:j])],Xvp[1,1:sum(N[1:j])])%*%t(c(XtfwdT[1,1:sum(N[1:j])],Xvp[1,1:sum(N[1:j])]))*gamma_cont #gamma tilde
    }
    else
    {
      mucont=sum(mu_sw[sum(N[1:j-1]+1):sum(N[1:j])])
      gcont=as.matrix(c(g_dtc[sum(N[1:j-1]+1):sum(N[1:j])],g_dvp[sum(N[1:j-1]+1):sum(N[1:j])]))
      gtcont=t(as.matrix(gcont*c(XtfwdT[1,sum(N[1:j-1]+1):sum(N[1:j])]*WFRcont[k,1],Xvp[1,sum(N[1:j-1]+1):sum(N[1:j])]*WFRcont[k,2])))
      VarCovXcont=cov(cbind(DeltaXtfwdt[,sum(N[1:j-1]+1):sum(N[1:j])],DeltaXvp[,sum(N[1:j-1]+1):sum(N[1:j])]))*(n/(n-1))
      gamma_cont=gamma_sw[(2*sum(N[1:j-1])+1):(2*sum(N[1:j])),(2*sum(N[1:j-1])+1):(2*sum(N[1:j]))]
      tgamma= c(XtfwdT[1,sum(N[1:j-1]+1):sum(N[1:j])],Xvp[1,sum(N[1:j-1]+1):sum(N[1:j])])%*%t(c(XtfwdT[1,sum(N[1:j-1]+1):sum(N[1:j])],Xvp[1,sum(N[1:j-1]+1):sum(N[1:j])]))*gamma_cont #gamma tilde
    }
    
    
    tgammavar=(tgamma%*%VarCovXcont)
    tgammavar2=(tgamma%*%VarCovXcont)%*%(tgamma%*%VarCovXcont)
    VaRContfrDGM[k,j]=(sum(diag(tgammavar))/2+mucont)-(sqrt(sum(diag(tgammavar2))/2+gtcont%*%VarCovXcont%*%t(gtcont)))*qnorm(alpha) #VaR Normal
    CVaRContfrDGM[k,j]=(sum(diag(tgammavar))/2+mucont)-(sqrt(sum(diag(tgammavar))/2+gtcont%*%VarCovXcont%*%t(gtcont)))*dnorm(qnorm(alpha))/(1-alpha)
    #VaR Cornish Fisher
    tgammavar3=tgammavar2%*%(tgamma%*%VarCovXcont)
    asimPmargc=sum(diag(tgammavar3))+3*gtcont%*%VarCovXcont%*%tgamma%*%VarCovXcont%*%t(gtcont) #gamma tercer momento
    asimPmargc=asimPmargc/(sqrt(sum(diag(tgammavar2))/2+gtcont%*%VarCovXcont%*%t(gtcont)))^3  #coef de asimetría
    VaRmargcstd=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc  #VaR estandarizado
    VaRContfrCF[k,j]=(sum(diag(tgammavar))/2+mucont)+(sqrt(sum(diag(tgammavar2))/2+gtcont%*%VarCovXcont%*%t(gtcont)))*VaRmargcstd #VaR Normal
    CVaRContfrCF[k,j]=(sum(diag(tgammavar))/2+sum(mucont))-(sqrt(sum(diag(tgammavar2))/2+gtcont%*%VarCovXcont%*%t(gtcont)))*dnorm(VaRmargcstd)/(pnorm(VaRmargcstd))
    
    #VaR Delta Normal
    VaRcontfr[k,j]=sum(mucont)-sqrt(gtcont%*%VarCovXcont%*%t(gtcont))*qnorm(alpha)
    CVaRcontfr[k,j]=sum(mucont)-sqrt(gtcont%*%VarCovXcont%*%t(gtcont))*dnorm(qnorm(alpha))/(1-alpha)
    
  }
}


print("Valor a nivel contrato")
print(V0_sw)
print("VaR a nivel contrato")
print(VaRcont)
print("VaR a nivel contrato CF")
print(VaRContCF)
print("VaR a nivel contrato DGN")
print(VaRContDGM)
print("CVaR a nivel contrato")
print(CVaRcont)
print("CVaR a nivel contrato CF")
print(CVaRContCF)
print("CVaR a nivel contrato DGN")
print(CVaRContDGM)

print("VaR a nivel factor de riesgo")
print(VaRcontfr)
print("VaR a nivel factor de riesgo CF")
print(VaRContfrCF)
print("VaR a nivel factor de riesgo DGN")
print(VaRContfrDGM)
print("CVaR a nivel factor de riesgo")
print(CVaRcontfr)
print("CVaR a nivel factor de riesgo CF")
print(CVaRContfrCF)
print("CVaR a nivel factor de riesgo DGN")
print(CVaRContfrDGM)

[1] "Valor a nivel contrato"
         [,1]     [,2]
[1,] 63.75999 28.39141
[1] "VaR a nivel contrato"
          [,1]       [,2]
[1,] -3.222905 -0.6542808
[1] "VaR a nivel contrato CF"
         [,1]       [,2]
[1,] -3.22345 -0.6542482
[1] "VaR a nivel contrato DGN"
          [,1]       [,2]
[1,] -3.223072 -0.6542632
[1] "CVaR a nivel contrato"
          [,1]       [,2]
[1,] -4.038086 -0.8188853
[1] "CVaR a nivel contrato CF"
          [,1]       [,2]
[1,] -4.038578 -0.8188548
[1] "CVaR a nivel contrato DGN"
          [,1]       [,2]
[1,] -4.038165 -0.8189138
[1] "VaR a nivel factor de riesgo"
           [,1]        [,2]
[1,] -3.2620188 -0.65605335
[2,] -0.1038407 -0.02335404
[1] "VaR a nivel factor de riesgo CF"
           [,1]        [,2]
[1,] -3.2625784 -0.65602061
[2,] -0.1042538 -0.02332571
[1] "VaR a nivel factor de riesgo DGN"
           [,1]        [,2]
[1,] -3.2621859 -0.65603573
[2,] -0.1040142 -0.02333644
[1] "CVaR a nivel factor de riesgo"
           [,1]        [,2]
[1,] -4.

###Riesgo a nivel total y total factor de riesgo 

In [ ]:
%%R
#RIESGO TOTAL SWAP

VarDeltaX_sw=cov(DeltaX_sw)*((n-1)/(n-2))
tgammavar_sw=(tgamma_sw%*%VarDeltaX_sw)
tgammavar2_sw=(tgammavar_sw)%*%(tgammavar_sw) 
#media
mudg_sw=ifelse((sum(diag(tgammavar_sw))/2+sum(mu_sw))>0,0,(sum(diag(tgammavar_sw))/2+mu_sw))

#desviación estándar
desv_sw=sqrt(sum(diag(tgammavar2_sw))/2+gt_sw%*%VarDeltaX_sw%*%t(gt_sw))

VaRT_sw_DGM=(mudg_sw)+(desv_sw)*qnorm(1-alpha)  #VaR Normal
CVaRT_sw_DGM=-(mudg_sw)-(desv_sw)*dnorm(qnorm(alpha))/(1-alpha)


###VaR Cornish Fisher
tgammavar3_sw=tgammavar2_sw%*%(tgammavar_sw)
asimP_sw=sum(diag(tgammavar3_sw))+3*gt_sw%*%VarDeltaX_sw%*%tgamma_sw%*%VarDeltaX_sw%*%t(gt_sw) #gamma tercer momento
asimP_sw=asimP_sw/(desv_sw)^3  #coef de asimetría
VaRstd_sw=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_sw  #VaR estandarizado
VaRT_sw_DGM_CF=(mudg_sw)+(desv_sw)*VaRstd_sw #VaR cf
CVaRT_sw_DGM_CF=-(mudg_sw)-(desv_sw)*dnorm(VaRstd_sw)/(pnorm(VaRstd_sw))

print(VaRT_sw_DGM)
print(CVaRT_sw_DGM)
print(VaRT_sw_DGM_CF)
print(CVaRT_sw_DGM_CF)


VaRTDN_fr_sw=matrix(0,1,2)
CVaRTDN_fr_sw=matrix(0,1,2)
VaRT_fr_sw_DGM=matrix(0,1,2)
CVaRT_fr_sw_DGM=matrix(0,1,2)
VaRT_fr_sw_DGM_CF=matrix(0,1,2)
CVaRT_fr_sw_DGM_CF=matrix(0,1,2)

WFRT=matrix(0,32,(m*32))

#RIESGO TOTAL POR FACTOR DE RIESGO DE FORWARDS DE TIPO DE CAMBIO

for (j in (1:32))
{
  WFRT[j,((j-1)*m+1):(j*m)]=matrix(1,1,m)    #Matriz de diseño para 
  WFRM=matrix(0,(m*32),(m*32))  #PASO CLAVE
  WFRM[((j-1)*m+1):(m*j),((j-1)*m+1):(m*j)]=matrix(1,m,m)  #PASO CLAVE    
  gamma_sw_fr=tgamma_sw*WFRM #evaluación de gamma multiplicado por x^{2}
  g_sw_fr=gt_sw*WFRT[j,]
  tgammavar_sw=(gamma_sw_fr%*%VarDeltaX_sw)
  tgammavar2_sw=(tgammavar_sw)%*%(tgammavar_sw) 
  mudg_sw=sum(diag(tgammavar_sw))/2+sum(mu_sw)
  #mudg_sw=0
  desv_sw=sqrt(sum(diag(tgammavar2_sw))/2+g_sw_fr%*%VarDeltaX_sw%*%t(g_sw_fr))
  VaRT_fr_sw_DGM[j]=(mudg_sw)+(desv_sw)*qnorm(1-alpha) #VaR Normal
  CVaRT_fr_sw_DGM[j]=-(mudg_sw)-(desv_sw)*dnorm(qnorm(alpha))/(1-alpha)
  
  #Delta Normal
  mudn_sw=0
  desvdn_sw=sqrt(g_sw_fr%*%VarDeltaX_sw%*%t(g_sw_fr))
  VaRTDN_fr_sw[j]=mudn_sw+desvdn_sw*qnorm(1-alpha) #VaR Normal por contrato Delta Normal
  CVaRTDN_fr_sw[j]=mudn_sw-desvdn_sw*(dnorm(qnorm(1-alpha))/(1-alpha))  #CVaR Normal por contrato Delta Normal
  
  
  ###VaR Cornish Fisher
  tgammavar3_sw=tgammavar2_sw%*%(tgammavar_sw)
  asimP_sw=sum(diag(tgammavar3_sw))+3*g_sw_fr%*%VarDeltaX_sw%*%gamma_sw_fr%*%VarDeltaX_sw%*%t(g_sw_fr) #gamma tercer momento
  asimP_sw=asimP_sw/(desv_sw)^3  #coef de asimetría
  
  VaRstd_sw=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_sw  #VaR estandarizado
  VaRT_fr_sw_DGM_CF[j]=(mudg_sw)+(desv_sw)*VaRstd_sw #VaR cf
  CVaRT_fr_sw_DGM_CF[j]=-(mudg_sw)-(desv_sw)*dnorm(VaRstd_sw)/(pnorm(VaRstd_sw))
  
}
print(VaRTDN_fr_sw)
print(CVaRTDN_fr_sw)
print(VaRT_fr_sw_DGM)
print(CVaRT_fr_sw_DGM)
print(VaRT_fr_sw_DGM_CF)
print(CVaRT_fr_sw_DGM_CF)

print(sum(V0_sw))


          [,1]
[1,] -1.996239
          [,1]
[1,] -2.501659
          [,1]
[1,] -1.996325
          [,1]
[1,] -2.501733
 [1] -1.352246e-01 -1.342734e-01 -1.308059e-01 -1.219632e-01 -2.088643e-01
 [6] -2.798355e-01 -3.297008e-01 -2.145093e-01 -2.513898e-01 -2.885225e-01
[11] -3.246897e-01 -1.019938e-01 -9.987182e-02 -9.728110e-02 -1.028766e-01
[16] -1.729341e-01 -5.546087e-05 -4.954040e-04 -1.272921e-03 -2.104863e-03
[21] -2.842145e-03 -3.723056e-03 -4.793234e-03 -4.720523e-03 -5.032656e-03
[26] -5.296435e-03 -5.508203e-03 -1.131542e-04 -6.220619e-04 -1.266838e-03
[31] -1.946418e-03 -2.588322e-03
 [1] -1.695771e-01 -1.683843e-01 -1.640359e-01 -1.529468e-01 -2.619243e-01
 [6] -3.509250e-01 -4.134581e-01 -2.690033e-01 -3.152530e-01 -3.618189e-01
[11] -4.071740e-01 -1.279043e-01 -1.252433e-01 -1.219944e-01 -1.290115e-01
[16] -2.168663e-01 -6.955017e-05 -6.212566e-04 -1.596294e-03 -2.639583e-03
[21] -3.564164e-03 -4.668862e-03 -6.010909e-03 -5.919727e-03 -6.311154e-03
[26] -6.641943e-03 -6.

## Riesgo de Opciones de tasa de interés

### Riesgo a nivel contrato y nivel contrato-factor de riesgo

In [ ]:
%%R


#Cálculo de matriz de pérdidas y ganancias FUTUROS TDC

#Cálculo de VaR por contrato
#creación de matrices de VaR, gamma, VaRCF por contrato
DeltaX_oir=DeltaX_port[,(sum(n_if[1:7],1)):(sum(n_if[1:8]))] 
m_oir=ncol(plazos_oir)
m=ncol(plazos_oir)

VaRmargc_oir=matrix(0,1,m) #Var por contrato Delta Normal
VaRmargcDN_oir=matrix(0,1,m) #Var por contrato Delta Normal 
VaRmargcstd_oir=matrix(0,1,m) #Var por contrato estandarizado 
VaRmargcCF_oir=matrix(0,1,m) #Var por contrato Cornish-Fisher
CVaRmargc_oir=matrix(0,1,m) #CVar por contrato estandarizado
CVaRmargcDN_oir=matrix(0,1,m) #Var por contrato Delta Normal 
CVaRmargcCF_oir=matrix(0,1,m) #CVar por contrato estandarizado

##factores riesgo
#Spot
VaRmargcDN_fr_oir=matrix(0,3,m) #Var por contrato Delta Normal
VaRmargc_fr_oir=matrix(0,3,m) #Var por contrato Delta Normal
VaRmargcDN_fr_oir=matrix(0,3,m) #Var por contrato Delta Normal
VaRmargcstd_fr_oir=matrix(0,3,m) #Var por contrato estandarizado 
VaRmargcCF_fr_oir=matrix(0,3,m) #Var por contrato Cornish-Fisher
CVaRmargcDN_fr_oir=matrix(0,3,m) #CVar por contrato estandarizado
CVaRmargc_fr_oir=matrix(0,3,m) #CVar por contrato estandarizado
CVaRmargcDN_fr_oir=matrix(0,3,m) #Var por contrato Delta Normal
CVaRmargcCF_fr_oir=matrix(0,3,m) #CVar por contrato estandarizado
WFR=diagv(c(1,1,1))
WFR=diagv(c(1,1,1))


for (i in 1:(m))
{
  delta_oir=t(delta_oir_T[seq(i,m*3,by=m)]) 
  tdelta_oir=t(delta_oir_T[seq(i,m*3,by=m)])*xp_oir[seq(i,m*3,by=m)]   #delta tilde
  gamma_oir=mgammaopc(x01[i],x02[i],K_oir[i],x03[i],plazos_oir[i],cp_oir[i],cs_oir,pr_oir,dct_oir,contratos_oir[i],nominal_oir)  #paso clave
  tgamma_oir=xp_oir[seq(i,m*3,by=m)]%*%t(xp_oir[seq(i,m*3,by=m)])*gamma_oir #gamma tilde
  Varmargcaux_oir=cov(DeltaX_oir[,seq(i,m*3,by=m)])*((n)/(n-1)) #matriz varcov factores de riesgo de cada contrato
  tgammavar_oir=(tgamma_oir%*%Varmargcaux_oir)
  tgammavar2_oir=(tgamma_oir%*%Varmargcaux_oir)%*%(tgamma_oir%*%Varmargcaux_oir)  
  mudg_oir=ifelse(sum(diag(tgammavar_oir))/2+mu_oir[i]>0,0,sum(diag(tgammavar_oir))/2+mu_oir[i])
  desv_oir=sqrt(sum(diag(tgammavar2_oir))/2+tdelta_oir%*%Varmargcaux_oir%*%t(tdelta_oir))

  mudn_oir=ifelse(mu_oir[i]>0,0,mu_oir[i])
  desvdn_oir=sqrt(tdelta_oir%*%Varmargcaux_oir%*%t(tdelta_oir))

  VaRmargcDN_oir[i]=mudn_oir+desvdn_oir*qnorm(1-alpha) #VaR Normal por contrato Delta Normal
  CVaRmargcDN_oir[i]=mudn_oir-desvdn_oir*(dnorm(qnorm(1-alpha))/(1-alpha))  #CVaR Normal por contrato Delta Normal

  VaRmargc_oir[i]=mudg_oir+desv_oir*qnorm(1-alpha) #VaR Normal por contrato
  CVaRmargc_oir[i]=mudg_oir-desv_oir*(dnorm(qnorm(1-alpha))/(1-alpha))
  
  tgammavar3_oir=tgammavar2_oir%*%(tgamma_oir%*%Varmargcaux_oir)
  asimPmargc_oir=sum(diag(tgammavar3_oir))+3*tdelta_oir%*%Varmargcaux_oir%*%tgamma_oir%*%Varmargcaux_oir%*%t(tdelta_oir) #tercer momento
  asimPmargc_oir=asimPmargc_oir/(desv_oir)^3  #coef de asimetría
  VaRmargcstd_oir[i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_oir  #VaR estandarizado
  VaRmargcCF_oir[i]=mudg_oir+desv_oir*VaRmargcstd_oir[i] 	    #VaR Cornish-Fisher
  CVaRmargcCF_oir[i]=mudg_oir-desv_oir*(dnorm(VaRmargcstd_oir[i])/(pnorm(VaRmargcstd_oir[i])))

#Por factor de riesgo y por contrato
#spot, tasa extranjera, tasa nacional
  for (j in 1:3)
  {
    WFRM=matrix(0,3,3)  #PASO CLAVE
    WFRM[j,j]=1  #PASO CLAVE
    delta_oir=t(delta_oir_T[seq(i,m*3,by=m)])*WFR[j,]
    tdelta_oir=t(delta_oir_T[seq(i,m*3,by=m)])*xp_oir[seq(i,m*3,by=m)]*WFR[j,]   #delta tilde
    gamma_oir=mgammaopc(x01[i],x02[i],K_oir[i],x03[i],plazos_oir[i],cp_oir[i],cs_oir,pr_oir,dct_oir,contratos_oir[i],nominal_oir)*WFRM #paso clave
   #print(WFRM)
    tgamma_oir=xp_oir[seq(i,m*3,by=m)]%*%t(xp_oir[seq(i,m*3,by=m)])*gamma_oir #gamma tilde
    Varmargcaux_oir=cov(DeltaX_oir[,seq(i,m*3,by=m)])*((n)/(n-1)) #matriz varcov factores de riesgo de cada contrato
    tgammavar_oir=(tgamma_oir%*%Varmargcaux_oir)
    tgammavar2_oir=(tgamma_oir%*%Varmargcaux_oir)%*%(tgamma_oir%*%Varmargcaux_oir)  
    #mudg_oir=sum(diag(tgammavar_oir))/2+mu_oir[i]
    mudg_oir=0
    desv_oir=sqrt(sum(diag(tgammavar2_oir))/2+tdelta_oir%*%Varmargcaux_oir%*%t(tdelta_oir))

    mudn_oir=0
    desvdn_oir=sqrt(tdelta_oir%*%Varmargcaux_oir%*%t(tdelta_oir))

    VaRmargcDN_fr_oir[j,i]=mudn_oir+desvdn_oir*qnorm(1-alpha) #VaR Normal por contrato Delta Normal
    CVaRmargcDN_fr_oir[j,i]=mudn_oir-desvdn_oir*(dnorm(qnorm(1-alpha))/(1-alpha))  #CVaR Normal por contrato Delta Normal

    VaRmargc_fr_oir[j,i]=mudg_oir+desv_oir*qnorm(1-alpha) #VaR Normal por contrato
    CVaRmargc_fr_oir[j,i]=mudg_oir-desv_oir*(dnorm(qnorm(1-alpha))/(1-alpha))
    
    tgammavar3_oir=tgammavar2_oir%*%(tgamma_oir%*%Varmargcaux_oir)
    asimPmargc_oir=sum(diag(tgammavar3_oir))+3*tdelta_oir%*%Varmargcaux_oir%*%tgamma_oir%*%Varmargcaux_oir%*%t(tdelta_oir) #tercer momento
    asimPmargc_oir=asimPmargc_oir/(desv_oir)^3  #coef de asimetría
    VaRmargcstd_fr_oir[j,i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_oir  #VaR estandarizado
    VaRmargcCF_fr_oir[j,i]=mudg_oir+desv_oir*VaRmargcstd_fr_oir[j,i] 	    #VaR Cornish-Fisher
    CVaRmargcCF_fr_oir[j,i]=mudg_oir-desv_oir*(dnorm(VaRmargcstd_fr_oir[j,i])/(pnorm(VaRmargcstd_fr_oir[j,i])))
  }



}


print("Valor a nivel contrato")
print(V0_oir)
print("VaR a nivel contrato Delta normal")
print(VaRmargcDN_oir)
print("VaR a nivel contrato Delta Gamma Normal")
print(VaRmargc_oir)
print("VaR a nivel contrato CF")
print(VaRmargcCF_oir)
print("CVaR a nivel contrato Delta normal")
print(CVaRmargcDN_oir)
print("CVaR a nivel contrato Delta Gamma Normal")
print(CVaRmargc_oir)
print("CVaR a nivel contrato CF")
print(CVaRmargcCF_oir)

print("VaR a nivel contrato delta normal")
print(VaRmargcDN_fr_oir)
print("VaR a nivel contrato Delta Gamma Normal")
print(VaRmargc_fr_oir)
print("VaR a nivel contrato CF")
print(VaRmargcCF_fr_oir)
print("CVaR a nivel contrato delta normal")
print(CVaRmargcDN_fr_oir)
print("CVaR a nivel contrato Delta Gamma Normal")
print(CVaRmargc_fr_oir)
print("CVaR a nivel contrato CF")
print(CVaRmargcCF_fr_oir)


[1] "Valor a nivel contrato"
         [,1]     [,2]
[1,] 2.215311 0.121693
[1] "VaR a nivel contrato Delta normal"
            [,1]        [,2]
[1,] -0.09461083 -0.03507321
[1] "VaR a nivel contrato Delta Gamma Normal"
            [,1]        [,2]
[1,] -0.09461224 -0.03508306
[1] "VaR a nivel contrato CF"
            [,1]        [,2]
[1,] -0.09451141 -0.03457494
[1] "CVaR a nivel contrato Delta normal"
           [,1]        [,2]
[1,] -0.1186458 -0.04398323
[1] "CVaR a nivel contrato Delta Gamma Normal"
           [,1]        [,2]
[1,] -0.1186476 -0.04399557
[1] "CVaR a nivel contrato CF"
           [,1]        [,2]
[1,] -0.1185607 -0.04355807
[1] "VaR a nivel contrato delta normal"
            [,1]          [,2]
[1,] -0.01842975 -0.0002750096
[2,] -0.08105373 -0.0143378489
[3,] -0.04436383 -0.0308265458
[1] "VaR a nivel contrato Delta Gamma Normal"
            [,1]          [,2]
[1,] -0.01842987 -0.0002750097
[2,] -0.08105373 -0.0143378489
[3,] -0.04436639 -0.0308375368
[1] "VaR a niv

### Riesgo a nivel total y total factor de riesgo

In [ ]:
%%R


#RIESGO TOTAL FORWARDS DE TIPO DE CAMBIO

VarDeltaX_oir=cov(DeltaX_oir)*(n/(n-1))

tgammavar_oir=(gammaT_oirT2%*%VarDeltaX_oir)
tgammavar2_oir=(tgammavar_oir)%*%(tgammavar_oir) 
mudg_oir=ifelse(sum(diag(tgammavar_oir))/2+sum(mu_oir)>0,0,sum(diag(tgammavar_oir))/2+sum(mu_oir))
desv_oir=sqrt(sum(diag(tgammavar2_oir))/2+deltaT_oir_T%*%VarDeltaX_oir%*%t(deltaT_oir_T))

VaRT_oir_DGM=(mudg_oir)+(desv_oir)*qnorm(1-alpha)  #VaR Normal
CVaRT_oir_DGM=(mudg_oir)-(desv_oir)*dnorm(qnorm(alpha))/(1-alpha)

  #Delta normal
  mudn_oir=ifelse(sum(mu_oir)>0,0,sum(mu_oir))
  desvdn_oir=sqrt(deltaT_oir_T%*%VarDeltaX_oir%*%t(deltaT_oir_T))
  VaRTDN_oir=mudn_oir+desvdn_oir*qnorm(1-alpha) #VaR Normal por contrato Delta Normal
  CVaRTDN_oir=mudn_oir-desvdn_oir*(dnorm(qnorm(1-alpha))/(1-alpha))  #CVaR Normal por contrato Delta Normal

###VaR Cornish Fisher
tgammavar3_oir=tgammavar2_oir%*%(tgammavar_oir)
asimP_oir=sum(diag(tgammavar3_oir))+3*deltaT_oir_T%*%VarDeltaX_oir%*%gammaT_oirT2%*%VarDeltaX_oir%*%t(deltaT_oir_T) #gamma tercer momento
asimP_oir=asimP_oir/(desv_oir)^3  #coef de asimetría
VaRstd_oir=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_oir  #VaR estandarizado
VaRT_oir_DGM_CF=(mudg_oir)+(desv_oir)*VaRstd_oir #VaR cf
CVaRT_oir_DGM_CF=(mudg_oir)-(desv_oir)*dnorm(VaRstd_oir)/(pnorm(VaRstd_oir))

print(VaRTDN_oir)
print(CVaRTDN_oir)
print(VaRT_oir_DGM)
print(CVaRT_oir_DGM)
print(VaRT_oir_DGM_CF)
print(CVaRT_oir_DGM_CF)

VaRTDN_fr_oir=matrix(0,1,3)
CVaRTDN_fr_oir=matrix(0,1,3)
VaRT_fr_oir_DGM=matrix(0,1,3)
CVaRT_fr_oir_DGM=matrix(0,1,3)
VaRT_fr_oir_DGM_CF=matrix(0,1,3)
CVaRT_fr_oir_DGM_CF=matrix(0,1,3)

WFRT=matrix(0,3,(m*3))
# WFRT[1,1:m]=matrix(1,1,m)
# WFRT[2,(m+1):(2*m)]=matrix(1,1,m)
# WFRT[3,(2*m+1):(3*m)]=matrix(1,1,m)
#print(WFRT)

#RIESGO TOTAL POR FACTOR DE RIESGO DE opciones
print(m)
for (j in (1:3))
{
WFRT[j,((j-1)*m+1):(j*m)]=matrix(1,1,m)    #Matriz de diseño para 
WFRM=matrix(0,(m*3),(m*3))  #PASO CLAVE
WFRM[((j-1)*m+1):(m*j),((j-1)*m+1):(m*j)]=matrix(1,m,m)  #PASO CLAVE  
#print(deltaT_oir_T)
#print(WFRM) 
#print(dim(WFRT))  
gamma_oirtT2_fr=gammaT_oirT2*WFRM #evaluación de gamma multiplicado por x^{2}

g_oirtT_fr=deltaT_oir_T*WFRT[j,]
# print(WFRM)
# print(g_oirtT_fr)
# print(gamma_oirtT2_fr)
# print(WFRT)
tgammavar_oir=(gamma_oirtT2_fr%*%VarDeltaX_oir)
tgammavar2_oir=(tgammavar_oir)%*%(tgammavar_oir) 
#mudg_oir=sum(diag(tgammavar_oir))/2+sum(mu_oir)
mudg_oir=0
desv_oir=sqrt(sum(diag(tgammavar2_oir))/2+g_oirtT_fr%*%VarDeltaX_oir%*%t(g_oirtT_fr))
VaRT_fr_oir_DGM[j]=(mudg_oir)+(desv_oir)*qnorm(1-alpha) #VaR Normal
CVaRT_fr_oir_DGM[j]=(mudg_oir)-(desv_oir)*dnorm(qnorm(alpha))/(1-alpha)

    #Delta Normal
    mudn_oir=0
    desvdn_oir=sqrt(g_oirtT_fr%*%VarDeltaX_oir%*%t(g_oirtT_fr))
    VaRTDN_fr_oir[j]=mudn_oir+desvdn_oir*qnorm(1-alpha) #VaR Normal por contrato Delta Normal
    CVaRTDN_fr_oir[j]=mudn_oir-desvdn_oir*(dnorm(qnorm(1-alpha))/(1-alpha))  #CVaR Normal por contrato Delta Normal

    

###VaR Cornish Fisher
tgammavar3_oir=tgammavar2_oir%*%(tgammavar_oir)
asimP_oir=sum(diag(tgammavar3_oir))+3*g_oirtT_fr%*%VarDeltaX_oir%*%gamma_oirtT2_fr%*%VarDeltaX_oir%*%t(g_oirtT_fr) #gamma tercer momento
 #print(asimP_oir)
asimP_oir=asimP_oir/(desv_oir)^3  #coef de asimetría
 
VaRstd_oir=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_oir  #VaR estandarizado
#print(VaRstd_oir)
VaRT_fr_oir_DGM_CF[j]=(mudg_oir)+(desv_oir)*VaRstd_oir #VaR cf
CVaRT_fr_oir_DGM_CF[j]=(mudg_oir)-(desv_oir)*dnorm(VaRstd_oir)/(pnorm(VaRstd_oir))
}

print(VaRTDN_fr_oir)
print(CVaRTDN_fr_oir)
print(VaRT_fr_oir_DGM)
print(CVaRT_fr_oir_DGM)
print(VaRT_fr_oir_DGM_CF)
print(CVaRT_fr_oir_DGM_CF)

print(sum(V0_oir))


# cbind(VaRTotal_fwtdc,sum(V0_fwtdc), VaRCont_fwtdc, V0_fwtdc)
# cbind(CVaRTotal_fwtdc,sum(V0_fwtdc), CVaRCont_fwtdc, V0_fwtdc)
# cbind(VaRTotal_fwtdc,VaRTotalfr1_fwtdc,VaRTotalfr2_fwtdc,VaRTotalfr3_fwtdc)
# cbind(CVaRTotal_fwtdc,CVaRTotalfr1_fwtdc,CVaRTotalfr2_fwtdc,CVaRTotalfr3_fwtdc)


           [,1]
[1,] -0.1142434
           [,1]
[1,] -0.1432658
           [,1]
[1,] -0.1142492
           [,1]
[1,] -0.1432731
          [,1]
[1,] -0.113876
           [,1]
[1,] -0.1429515
[1] 2
            [,1]        [,2]        [,3]
[1,] -0.01864056 -0.08466009 -0.06791784
            [,1]       [,2]        [,3]
[1,] -0.02337602 -0.1061672 -0.08517171
            [,1]        [,2]        [,3]
[1,] -0.01864068 -0.08466009 -0.06792671
            [,1]       [,2]        [,3]
[1,] -0.02337616 -0.1061672 -0.08518284
            [,1]       [,2]        [,3]
[1,] -0.01859224 -0.0846589 -0.06717685
            [,1]       [,2]        [,3]
[1,] -0.02333442 -0.1061661 -0.08453703
[1] 2.337004


#Riesgo Integral

In [ ]:
%%R
#Cálculo de matriz de varianza covarianza de todos los factores de riesgo del PORTAFOLIO
VarDeltaX_port=as.matrix(cov(DeltaX_port)*(n-1)/(n-2))

mu_port=as.matrix(cbind(mu_acc_div,mu_bcc,mu_bm,mu_bd,mu_ftdc,mu_find,mu_sw,mu_oir)) #derivada con respecto al tiempo
#INTEGRACIÓN DE PRIMERAS DERIVADAS (GRADIENTE)
gt_port=as.matrix(cbind(g_acc_divt,g_bcct,g_bmt,g_bdt,g_ftdctT,g_findtT,gt_sw,deltaT_oir_T)) #derivada con respecto las X's
#validar que ncol(X_port)=ncol(gt_port)

#INTEGRACIÓN DE MATRIZ HESSIANA

gammat_port=matrix(0,sum(n_if),sum(n_if))
gammat_port[1:n_if[1],1:n_if[1]]=gamma_acc_divt
gammat_port[(sum(n_if[1:1])+1):(sum(n_if[1:2])),(sum(n_if[1:1])+1):(sum(n_if[1:2]))]=gamma_bcct2
gammat_port[(sum(n_if[1:2])+1):(sum(n_if[1:3])),(sum(n_if[1:2])+1):(sum(n_if[1:3]))]=gamma_bmt2
gammat_port[(sum(n_if[1:3])+1):(sum(n_if[1:4])),(sum(n_if[1:3])+1):(sum(n_if[1:4]))]=gamma_bdt2
gammat_port[(sum(n_if[1:4])+1):(sum(n_if[1:5])),(sum(n_if[1:4])+1):(sum(n_if[1:5]))]=gamma_ftdctT2
gammat_port[(sum(n_if[1:5])+1):(sum(n_if[1:6])),(sum(n_if[1:5])+1):(sum(n_if[1:6]))]=gamma_findtT2
gammat_port[(sum(n_if[1:6])+1):(sum(n_if[1:7])),(sum(n_if[1:6])+1):(sum(n_if[1:7]))]=tgamma_sw
gammat_port[(sum(n_if[1:7])+1):(sum(n_if[1:8])),(sum(n_if[1:7])+1):(sum(n_if[1:8]))]=gammaT_oirT2



#CÁLCULO DE VAR Y CVAR A NIVEL PORTAFOLIO
tgammavar_port=(gammat_port%*%VarDeltaX_port)
tgammavar2_port=(tgammavar_port)%*%(tgammavar_port) 
mu_portT=ifelse((sum(diag(tgammavar_port))/2+sum(mu_port))>0,0,(sum(diag(tgammavar_port))/2+sum(mu_port)))
desv_portT=(sqrt(sum(diag(tgammavar2_port))/2+gt_port%*%VarDeltaX_port%*%t(gt_port)))
VaRT_port_DGM=mu_portT+desv_portT*qnorm(1-alpha) #VaR Normal
CVaRT_port_DGM=mu_portT-desv_portT*dnorm(qnorm(alpha))/(1-alpha)

tgammavar2_port[(sum(n_if[1:5])+1):(sum(n_if[1:6])),(sum(n_if[1:5])+1):(sum(n_if[1:6]))]

###VaR Cornish Fisher
tgammavar3_port=tgammavar2_port%*%(tgammavar_port)
asimP_port=sum(diag(tgammavar3_port))+3*gt_port%*%VarDeltaX_port%*%gammat_port%*%VarDeltaX_port%*%t(gt_port) #gamma tercer momento
asimP_port=asimP_port/(sqrt(sum(diag(tgammavar2_port))/2+gt_port%*%VarDeltaX_port%*%t(gt_port)))^3  #coef de asimetría
VaRstd_port=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_port  #VaR estandarizado
VaRT_port_DGM_CF=mu_portT+desv_portT*VaRstd_port #VaR cf
CVaRT_port_DGM_CF=mu_portT-desv_portT*dnorm(VaRstd_port)/(pnorm(VaRstd_port))


VaRT_port_DGM/V0T_port
CVaRT_port_DGM/V0T_port
VaRT_port_DGM_CF/V0T_port
CVaRT_port_DGM_CF/V0T_port


Streaming output truncated to the last 5000 lines.
V78          1.677541e-05  1.912202e-05  1.720982e-05  1.443765e-05
V1           2.192091e-06  2.026722e-06  1.727952e-06  1.404528e-06
V2          -3.496570e-07 -2.435174e-07 -2.655723e-07 -3.150480e-07
V3          -3.539142e-07 -2.441799e-07 -2.670533e-07 -3.182754e-07
V1          -4.251237e-07  2.444002e-07 -9.602919e-07 -2.534606e-06
V2           0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
V3           1.538184e-05  1.173015e-05  9.268224e-06  7.091931e-06
V1           1.025242e-06  1.729380e-06  8.140498e-07 -4.280694e-07
V2           1.025242e-06  1.729380e-06  8.140498e-07 -4.280694e-07
V3           1.018915e-06  1.718655e-06  8.089970e-07 -4.254183e-07
V4           1.012610e-06  1.707968e-06  8.039618e-07 -4.227763e-07
V5           1.006342e-06  1.697345e-06  7.989572e-07 -4.201504e-07
V6           1.000118e-06  1.686796e-06  7.939873e-07 -4.175427e-07
V7           9.939350e-07  1.676317e-06  7.890508e-07 -4.149524e-

R[write to console]: Error in gammat_port %*% VarDeltaX_port : non-conformable arguments
Calls: <Anonymous> -> <Anonymous> -> withVisible




Error in gammat_port %*% VarDeltaX_port : non-conformable arguments
Calls: <Anonymous> -> <Anonymous> -> withVisible
